In [4]:
# @title Env
!pip install cartopy
!pip install torch_geometric
!pip install torchcde
!pip install PyWavelets

from google.colab import drive

import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import argparse
import yaml
import torch
import os
import argparse
import yaml
from pathlib import Path
import json
import sys
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import torch.nn.functional as F
from typing import Any, Callable, Tuple, Union
from torch import nn, optim
from torch.optim import Adam
import networkx as nx
import random
from torch.autograd import Variable
import gc
import torchcde
import math
from statsmodels.tsa.seasonal import STL
from tqdm import tqdm
import pickle
import logging
import time
import pywt

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/CLCRN-main')

# !pip install torch_scatter
# !mkdir -p /content/drive/MyDrive/colab_packages
# !cp -r /usr/local/lib/python3.*/dist-packages/torch_scatter /content/drive/MyDrive/colab_packages/
!cp -r /content/drive/MyDrive/colab_packages/torch_scatter /usr/local/lib/python3.*/dist-packages/  # Wheels have been built and saved in 'My Drive/colab_packages'

from supervisor import Supervisor

dataName = 'tmp2m'
dim = 0
if dataName == 'tmp2m':
    mean = 2.839279e+02
    std = 3.833667e+01
    max_std = (3.145732e+02 - mean) / std
    min_std = (2.325919e+02 - mean) / std
    p99 = (304.43316650390625 - mean) / std
    p95 = (301.278076171875 - mean) / std
    p90 = (298.9938659667969 - mean) / std
    p75 = (2.934141e+02 - mean) / std
    p50 = (2.853937e+02 - mean)/std
    p25 = (2.754702e+02 - mean) / std

## Specify configuration for adversatial target construction
caseStudy = 0

## Specify configuration for FABLE
padding = 'values'                                                              # ['values']
stepNum = 500
implementations = 1
clampEpsilon = 2.5                                                              # [0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]

lr = 0.01
batch_size = 48
seq_length = 12
H = 40
W = 30
device = 'cuda'
visualize = 1
test_sample_num = 0
use_optimizer = 'Adam' # ['Adam', 'PGD', 'pure']
reg = 1

wavelet_decomposition_dimension = 3
if wavelet_decomposition_dimension == 2:
    target_key = ['ad', 'da', 'dd']                                             # For 2d transform, ['aa', 'ad', 'da', 'dd']
    if reg == 1:
        coeff_weights = {'LH': 0.3, 'HL': 0.3,'HH': 0.5}
        reg_p = 2
        lambda_reg = 0.000005
elif wavelet_decomposition_dimension == 3:
    target_key = ['LLH', 'LHL', 'LHH', 'HLL', 'HLH', 'HHL', 'HHH']                     # For 3d transform, ['LLL', 'LLH', 'LHL', 'LHH', 'HLL', 'HLH', 'HHL', 'HHH']
    if reg == 1:
        # coeff_weights = {'LLH': 0.9, 'LHL': 0.9, 'LHH': 0.5, 'HLH': 0.5, 'HHL': 0.5, 'HHH': 0.3}
        coeff_weights = {'LLH': 0.8, 'LHL': 0.8,  'HLL': 0.8, 'LHH': 0.5, 'HLH': 0.5, 'HHL': 0.5, 'HHH': 0.2}
        reg_p = 2
        lambda_reg = 0.000003

fix = 'ablation_1'

best_CLCRN_model_epoch = 84

Mounted at /content/drive


In [ ]:
# @title FABLE: Based on 2d Wavelet Transform
!pip install PyWavelets

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import gc
import yaml
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import pickle
import torch
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def wavelet_transform_3d(data_3d, wavelet='haar'):
    """
    data_3d: shape (Time, H, W)
    Returns a dict with keys {'aa','ad','da','dd'} after single-level transform.
    """
    coeffs = pywt.dwtn(data_3d, wavelet=wavelet, axes=(1, 2))
    return coeffs

def wavelet_inverse_transform_3d(coeffs, wavelet='haar'):
    """
    Inverse of wavelet_transform_3d. Returns shape (Time, H, W).
    """
    reconstructed_data = pywt.idwtn(coeffs, wavelet=wavelet, axes=(1, 2))
    return reconstructed_data

class FABLE:
    def __init__(self, forward_func, wavelet, level, lons, lats):
        self.forward_func = forward_func
        self.wavelet = wavelet
        self.level = level
        self.lons = lons
        self.lats = lats

        self.lr = lr

    def rmse(self, pred, target):
        loss = F.mse_loss(pred, target)
        return torch.sqrt(loss)

    def wavelet_decompose_3d(self, data_3d):
        return wavelet_transform_3d(data_3d, wavelet=self.wavelet)

    def wavelet_reconstruct_3d(self, coeffs_dict):
        return wavelet_inverse_transform_3d(coeffs_dict, wavelet=self.wavelet)

    def create_regular_padded(self, lons, lats, values, fill_value=0.0):
        unique_lons = sorted(list(set(lons)))
        unique_lats = sorted(list(set(lats)))
        W = len(unique_lons)
        H = len(unique_lats)

        padded_data = np.full((H, W), fill_value, dtype=np.float32)             # '(0,0)' start from 'left-bottom' in the padded rectangle
        lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
        lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}

        for lon, lat, val in zip(lons, lats, values):
            ix = lon_to_x[lon]
            iy = lat_to_y[lat]
            padded_data[iy, ix] = val                                           # So, it should be (iy, ix) since '(0,0)', which starts from 'left-bottom' in the padded rectangle, should be converted to start from 'left-top'

        return padded_data, unique_lons, unique_lats

    def _forward_loss_given_params(
        self, param_vector,
        original_inputs,
        x_indices, y_indices,
        batch_size, seq_length,
        clampEpsilon, mean, std,
        target_torch, device
    ):
        """
        Directly reconstruct X from param_vector and compute the loss with optimized unpad operation.
        """

        S1 = param_vector[:,:,0,:,:] + param_vector[:,:,1,:,:]
        S2 = param_vector[:,:,2,:,:] + param_vector[:,:,3,:,:]
        D1 = param_vector[:,:,0,:,:] - param_vector[:,:,1,:,:]
        D2 = param_vector[:,:,2,:,:] - param_vector[:,:,3,:,:]

        reconstructed_inputs = torch.zeros(
            (batch_size, seq_length, param_vector[:,:,0,:,:].shape[2]*2, param_vector[:,:,0,:,:].shape[3]*2),
            device=param_vector.device,
            dtype=param_vector.dtype
        )

        reconstructed_inputs[:, :, ::2, ::2]   = (S1 + S2) / 2.0
        reconstructed_inputs[:, :, 1::2, ::2]  = (S1 - S2) / 2.0
        reconstructed_inputs[:, :, ::2, 1::2]  = (D1 + D2) / 2.0
        reconstructed_inputs[:, :, 1::2, 1::2] = (D1 - D2) / 2.0

        # Restore the input with valid locations
        reconstructed_inputs = reconstructed_inputs[:, :, y_indices, x_indices].unsqueeze(-1).permute(1, 0, 2, 3)

        # Clip the reconstructed inputs to be within bounds
        lower_bound = torch.maximum(original_inputs - clampEpsilon, torch.tensor(min_std, device = original_inputs.device))
        upper_bound = torch.minimum(original_inputs + clampEpsilon, torch.tensor(max_std, device = original_inputs.device))
        perturbed_inputs = torch.clamp(reconstructed_inputs, lower_bound, upper_bound)

        # Forward pass and calculate loss
        perturbed_inputs = perturbed_inputs.to(device)
        forecast = self.forward_func(perturbed_inputs)  # Shape: (seq_length, batch_size, ...)
        loss_val = self.rmse(forecast, target_torch)

        return perturbed_inputs, loss_val

    def calculate_regularization_loss(self, perturbed_param_vector, original_param_vector, coeff_weights):
        reg_loss = 0
        key_to_index = {'LL':0, 'LH': 1, 'HL': 2, 'HH': 3}                      # Coefficients indices
        for key, weight in coeff_weights.items():
            index = key_to_index[key]
            diff = (perturbed_param_vector[:, :, index, :, :] - original_param_vector[:, :, index, :, :])
            reg_loss += weight * torch.norm(diff, p=reg_p)

        return reg_loss

    def perturb(self, inputs, target, clampEpsilon, step_num, mean, std):
        """
        Perturbs the inputs based on gradients computed on wavelet coefficients.
        inputs.shape: (seq_length, batch_size, locations, 1)
        """
        device = inputs.device
        self.forward_func.eval()

        seq_length, batch_size, locations, variables = inputs.shape

        ulonlat_record = []
        padded_data_batch = []

        # Loop over each batch to process all time steps
        for b_idx in range(batch_size):
            time_step_padded = []
            time_step_ulonlat = []
            for t in range(seq_length):                                         # Process each time step
                data_1d = inputs[t, b_idx, :, 0].detach().cpu().numpy()         # Extract (locations,) for this time step and batch
                padded_data, unique_lons, unique_lats = self.create_regular_padded(self.lons, self.lats, data_1d, fill_value=0.0)
                time_step_padded.append(padded_data)                            # Store padded data for this time step
                time_step_ulonlat.append((unique_lons, unique_lats))            # Record corresponding lon/lat info
            padded_data_batch.append(np.stack(time_step_padded, axis=0))        # Shape: (seq_length, H, W) for this batch
            ulonlat_record.append(time_step_ulonlat)

        padded_data_batch = np.stack(padded_data_batch, axis=0)                 # Shape: (batch_size, seq_length, H, W)

        # Record the valid indices in the tensor 'padded_data_batch'
        unique_lons, unique_lats = ulonlat_record[0][0]                         # Use mapping from the first batch and time step
        lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
        lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}
        valid_indices = [
            (lon_to_x[lon], lat_to_y[lat])
            for lon, lat in zip(self.lons, self.lats)
            if lon in lon_to_x and lat in lat_to_y
        ]
        x_indices, y_indices = zip(*valid_indices)
        x_indices = torch.tensor(x_indices, dtype=torch.long, device=device)
        y_indices = torch.tensor(y_indices, dtype=torch.long, device=device)

        # 'pywt.dwtn' can implement Wavelet Transform along side any dimensions, where here axes = (2, 3) corresponds to the dimensions of H and W for each (batch, seq_length)
        coeffs_list = pywt.dwtn(padded_data_batch, wavelet = 'Haar', axes = (2, 3))  # coeffs_list['xx'].shape: (32, 7, 14, 29)

        # Create param_vector directly as a tensor of shape (batch_size, seq_length, 4, H/2, W/2), here, e.g., torch.Size([32, 7, 4, 14, 29])
        # //MARK (yue, 1/18/2025): the 'param_vector' is correct as it can be used to reconstruct the Original X, as I tested
        param_vector = torch.zeros((batch_size, seq_length, len(coeffs_list), coeffs_list['aa'].shape[2], coeffs_list['aa'].shape[3]), dtype=torch.float32, device=device)
        param_vector[:,:,0,:,:] = torch.tensor(coeffs_list['aa'], dtype=torch.float32).detach()
        param_vector[:,:,1,:,:] = torch.tensor(coeffs_list['ad'], dtype=torch.float32).detach()
        param_vector[:,:,2,:,:] = torch.tensor(coeffs_list['da'], dtype=torch.float32).detach()
        param_vector[:,:,3,:,:] = torch.tensor(coeffs_list['dd'], dtype=torch.float32).detach()

        perturbed_param_vector = param_vector.detach().clone()
        perturbed_param_vector.requires_grad_(True)
        target_torch = target.clone().detach().to(device)

        key_to_index = {'aa': 0, 'ad': 1, 'da': 2, 'dd': 3}
        target_indices = [key_to_index[key] for key in target_key]
        non_target_indices = [i for i in range(4) if i not in target_indices]
        unchanged_param = param_vector[:, :, non_target_indices, :, :].detach().clone()

        if use_optimizer == 'Adam':
            optimizer = torch.optim.Adam([perturbed_param_vector], lr = lr)

        best_loss = float('inf')
        for epoch in range(step_num):
            optimizer.zero_grad()

            perturbed_inputs, loss = self._forward_loss_given_params(
                perturbed_param_vector,
                inputs,
                x_indices, y_indices,
                batch_size=batch_size,
                seq_length=seq_length,
                clampEpsilon=clampEpsilon,
                mean=mean,
                std=std,
                target_torch=target_torch,
                device=device
            )

            ### regularizer
            if reg == 1:
                reg_loss = self.calculate_regularization_loss(perturbed_param_vector, param_vector, coeff_weights)
                total_loss = loss + lambda_reg * reg_loss
                print(f"Epoch {epoch} | Total Loss: {total_loss.item():.6f}, Loss: {loss.item():.6f}, reg_loss: {reg_loss:.6f}")
                total_loss.backward()
            else:
                print(f"Epoch {epoch} | Loss: {loss.item():.6f}")
                loss.backward()

            if perturbed_param_vector.grad is not None:
                perturbed_param_vector.grad[:, :, non_target_indices, :, :] = 0

            optimizer.step()

            with torch.no_grad():
                perturbed_param_vector[:, :, non_target_indices, :, :] = unchanged_param

            # print(torch.sum((perturbed_param_vector - param_vector)== 0).item() / param_vector.numel())

            if loss.item() < best_loss:
                best_loss = loss.item()
                best_perturbed_inputs = perturbed_inputs


        return best_perturbed_inputs

def main():
    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]
    with open("experiments/config_clcrn_tmp2m.yaml", 'r') as f:
        clcrnConfig = yaml.load(f, Loader=yaml.SafeLoader)
    clcrnModelTrained = Supervisor(**clcrnConfig)
    clcrnModelTrained.load_model(best_CLCRN_model_epoch)
    valIterator = clcrnModelTrained._data['{}_loader'.format('test')]

    fable = FABLE(forward_func=clcrnModelTrained.model, wavelet='haar', level=1, lons=lons, lats=lats)

    data_list = []
    breakFlag = 0
    for N in range(implementations):
        print(f'Random Implementation {N}')
        if caseStudy == 0:
            predList = torch.load(f'predList_tmp2m_{N}.pth')
            targetList = torch.load(f'targetList_tmp2m_{N}.pth')
        elif caseStudy == 1:
            predList = torch.load('case_predList_tmp2m.pth')
            targetList = torch.load('case_targetList_tmp2m.pth')

        for idx, (x, y) in enumerate(valIterator):
            if x.shape[0] != batch_size:
                break

            print(idx)
            # if idx == 1:
            #     break

            thisX = x.permute(1, 0, 2, 3).to('cuda')
            clcrnModelTrained.model.eval()

            yPred = predList[idx].permute(1, 0, 2, 3).to('cuda')
            yTarget = targetList[idx].permute(1, 0, 2, 3).to('cuda')

            xPerturbed = fable.perturb(thisX, yTarget, clampEpsilon, stepNum, mean, std)  # thisX.shape: (time_step, batch_size, locations, 1), e.g., [7, 32, 1320, 1]
            yPerturbed = clcrnModelTrained.model(xPerturbed.to('cuda'))

            for b in range(x.shape[0]):
                data_list.append({
                    "thisX": thisX[:, b, :, :].detach().cpu().numpy().tolist(),
                    "yPred": yPred[:, b, :, :].detach().cpu().numpy().tolist(),
                    "yTarget": yTarget[:, b, :, :].detach().cpu().numpy().tolist(),
                    "xPerturbed": xPerturbed[:, b, :, :].detach().cpu().numpy().tolist(),
                    "yPerturbed": yPerturbed[:, b, :, :].detach().cpu().numpy().tolist(),
                })

            df = pd.DataFrame(data_list)
            if caseStudy == 0:
                df.to_csv(f"results/FABLE_{dataName}_{clampEpsilon}_{stepNum}_2d_{fix}_{implementations}.csv",index=False)
            else:
                df.to_csv(f"results/FABLE_{dataName}_{clampEpsilon}_{stepNum}_2d_{fix}_caseStudy_{implementations}.csv", index=False)

            if caseStudy == 1:
                testBatchNum = 0
                if breakFlag == testBatchNum:
                    break
                breakFlag += 1

            del thisX, yPred, yTarget, xPerturbed, yPerturbed
            torch.cuda.empty_cache()
            gc.collect()

if __name__ == "__main__":
    main()



In [5]:
# @title FABLE: Based on 3d Wavelet Transform
!pip install PyWavelets

!pip install PyWavelets

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import gc
import yaml
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import pickle
import torch
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def haar_3d_inverse(LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH):
    """
    Single-level 3D Haar inverse transform.
    Each sub-band has shape (B, T2, H2, W2).
    The output will have shape (B, T, H, W),
    where T=2*T2, H=2*H2, W=2*W2.
    """
    B, T2, H2, W2 = LLL.shape
    T = T2 * 2
    H = H2 * 2
    W = W2 * 2

    inv_sqrt8 = 1.0 / math.sqrt(8.0)
    x_recon = torch.zeros((B, T, H, W), device=LLL.device, dtype=LLL.dtype)

    x_recon[:, ::2, ::2, ::2]   = (LLL + LLH + LHL + LHH + HLL + HLH + HHL + HHH) * inv_sqrt8
    x_recon[:, ::2, ::2, 1::2]  = (LLL - LLH + LHL - LHH + HLL - HLH + HHL - HHH) * inv_sqrt8
    x_recon[:, ::2, 1::2, ::2]  = (LLL + LLH - LHL - LHH + HLL + HLH - HHL - HHH) * inv_sqrt8
    x_recon[:, ::2, 1::2, 1::2] = (LLL - LLH - LHL + LHH + HLL - HLH - HHL + HHH) * inv_sqrt8

    x_recon[:, 1::2, ::2, ::2]  = (LLL + LLH + LHL + LHH - HLL - HLH - HHL - HHH) * inv_sqrt8
    x_recon[:, 1::2, ::2, 1::2] = (LLL - LLH + LHL - LHH - HLL + HLH - HHL + HHH) * inv_sqrt8
    x_recon[:, 1::2, 1::2, ::2] = (LLL + LLH - LHL - LHH - HLL - HLH + HHL + HHH) * inv_sqrt8
    x_recon[:, 1::2, 1::2, 1::2] = (LLL - LLH - LHL + LHH - HLL + HLH + HHL - HHH) * inv_sqrt8

    # x_recon.shape: torch.Size([32, 8, 28, 58])

    return x_recon

def wavelet_decompose_3d_pywt(x_4d):
    """
    Single-level 3D wavelet transform with pywt.dwtn along (T,H,W).
    x_4d shape: (B, T, H, W).
    Returns dict of 8 sub-bands: 'LLL','LLH','LHL','LHH','HLL','HLH','HHL','HHH'.
    Each sub-band has shape (B, T/2, H/2, W/2).
    """
    x_np = x_4d.detach().cpu().numpy()                                          # Convert to numpy if needed
    # PyWavelets forward 3D transform
    coeffs_dict = pywt.dwtn(x_np, wavelet='haar', axes=(1,2,3))
    # Convert back to torch
    LLL = torch.tensor(coeffs_dict['aaa'], device=x_4d.device, dtype=x_4d.dtype)
    LLH = torch.tensor(coeffs_dict['aad'], device=x_4d.device, dtype=x_4d.dtype)
    LHL = torch.tensor(coeffs_dict['ada'], device=x_4d.device, dtype=x_4d.dtype)
    LHH = torch.tensor(coeffs_dict['add'], device=x_4d.device, dtype=x_4d.dtype)
    HLL = torch.tensor(coeffs_dict['daa'], device=x_4d.device, dtype=x_4d.dtype)
    HLH = torch.tensor(coeffs_dict['dad'], device=x_4d.device, dtype=x_4d.dtype)
    HHL = torch.tensor(coeffs_dict['dda'], device=x_4d.device, dtype=x_4d.dtype)
    HHH = torch.tensor(coeffs_dict['ddd'], device=x_4d.device, dtype=x_4d.dtype)

    return {'LLL': LLL, 'LLH': LLH, 'LHL': LHL, 'LHH': LHH, 'HLL': HLL, 'HLH': HLH, 'HHL': HHL, 'HHH': HHH}


class FABLE:
    def __init__(self, forward_func, wavelet, level, lons, lats):
        self.forward_func = forward_func
        self.wavelet = wavelet
        self.level = level
        self.lons = lons
        self.lats = lats
        self.lr = lr

    def rmse(self, pred, target):
        loss = F.mse_loss(pred, target)
        return torch.sqrt(loss)

    def wavelet_decompose_3d(self, data_4d):
        return wavelet_decompose_3d_pywt(data_4d)

    def wavelet_reconstruct_3d(self, coeffs_dict):
        return haar_3d_inverse(
            coeffs_dict['LLL'], coeffs_dict['LLH'], coeffs_dict['LHL'], coeffs_dict['LHH'],
            coeffs_dict['HLL'], coeffs_dict['HLH'], coeffs_dict['HHL'], coeffs_dict['HHH']
        )

    def create_regular_padded(self, lons, lats, values, fill_value=0.0):
        unique_lons = sorted(list(set(lons)))
        unique_lats = sorted(list(set(lats)))
        W = len(unique_lons)
        H = len(unique_lats)

        padded_data = np.full((H, W), fill_value, dtype=np.float32)
        lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
        lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}

        for lon, lat, val in zip(lons, lats, values):
            ix = lon_to_x[lon]
            iy = lat_to_y[lat]
            padded_data[iy, ix] = val

        return padded_data, unique_lons, unique_lats

    def clamp_data(self, perturbed_data, original_data, clampEpsilon, mean, std):
        lower_bound = torch.maximum(original_data - clampEpsilon, torch.tensor(min_std, device=device))
        upper_bound = torch.minimum(original_data + clampEpsilon, torch.tensor(max_std, device=device))

        return torch.clamp(perturbed_data, lower_bound, upper_bound)

    def calculate_regularization_loss(self, perturbed_param_vector, original_param_vector, coeff_weights):
        reg_loss = 0
        key_to_index = {
            'LLL': 0, 'LLH': 1, 'LHL': 2, 'LHH': 3,
            'HLL': 4, 'HLH': 5, 'HHL': 6, 'HHH': 7
        }
        for key, weight in coeff_weights.items():
            index = key_to_index[key]
            diff = perturbed_param_vector[:, index, :, :, :] - original_param_vector[:, index, :, :, :]
            reg_loss += weight * torch.norm(diff, p=reg_p)
        return reg_loss

    def _forward_loss_given_params(
        self, param_vector,
        original_inputs,
        x_indices, y_indices,
        batch_size, seq_length,
        clampEpsilon, mean, std,
        target_torch, device
    ):
        """
        param_vector shape: (B, 8, T/2, H/2, W/2).
        Reconstruct with manual 3D inverse Haar.
        Map (B, T, H, W) -> (T, B, locations, 1), clamp, then compute loss.
        """
        # Build a dict of sub-bands
        coeffs_dict = {
            'LLL': param_vector[:, 0, :, :, :],
            'LLH': param_vector[:, 1, :, :, :],
            'LHL': param_vector[:, 2, :, :, :],
            'LHH': param_vector[:, 3, :, :, :],
            'HLL': param_vector[:, 4, :, :, :],
            'HLH': param_vector[:, 5, :, :, :],
            'HHL': param_vector[:, 6, :, :, :],
            'HHH': param_vector[:, 7, :, :, :],
        }

        reconstructed_4d = self.wavelet_reconstruct_3d(coeffs_dict)             # Reconstruct: shape (B, T, H, W)
        recon_perm = reconstructed_4d.permute(1, 0, 2, 3)                       # Permute to (T, B, H, W)

        # Flatten so we can gather
        width = recon_perm.shape[-1]                                            # recon_perm: (T,B,H,W)
        recon_2d = recon_perm.reshape(seq_length, batch_size, -1)               # shape => (T,B,H*W)

        # Gather only the valid indices
        linear_idx = y_indices * width + x_indices
        perturbed_inputs_2d = torch.gather(
            recon_2d, dim=2,
            index=linear_idx.unsqueeze(0).unsqueeze(0).expand(seq_length, batch_size, -1)
        )                                                                       # (T,B,locations)
        perturbed_inputs_4d = perturbed_inputs_2d.unsqueeze(-1)                 # (T,B,locations,1)

        # Clamp
        perturbed_inputs_4d = self.clamp_data(perturbed_inputs_4d, original_inputs, clampEpsilon, mean, std)

        # Forward pass
        forecast = self.forward_func(perturbed_inputs_4d.to(device))
        loss_val = self.rmse(forecast, target_torch)
        # print(forecast.shape, target_torch.shape)
        loss_val = self.rmse(forecast, target_torch)
        return perturbed_inputs_4d, loss_val

    def perturb(self, inputs, target, clampEpsilon, step_num, mean, std):
        """
        inputs: (T,B,locations,1)
        target: (T,B,locations,1)
        """
        device = inputs.device
        self.forward_func.eval()

        seq_length, batch_size, locations, _ = inputs.shape

        # Example: record lon/lat for each time step in each batch
        ulonlat_record = []
        padded_data_batch = []

        # Build a (B,T,H,W) data array
        for b_idx in range(batch_size):
            time_step_padded = []
            time_step_ulonlat = []
            for t in range(seq_length):
                data_1d = inputs[t, b_idx, :, 0].detach().cpu().numpy()
                padded_data, unique_lons, unique_lats = self.create_regular_padded(
                    self.lons, self.lats, data_1d, fill_value=0.0
                )
                time_step_padded.append(padded_data)
                time_step_ulonlat.append((unique_lons, unique_lats))
            padded_data_batch.append(np.stack(time_step_padded, axis=0))
            ulonlat_record.append(time_step_ulonlat)

        padded_data_batch = np.stack(padded_data_batch, axis=0)  # (B, T, H, W)
        padded_data_batch_torch = torch.tensor(padded_data_batch, dtype=torch.float32, device=device)

        # Record the valid indices in the tensor 'padded_data_batch'
        unique_lons, unique_lats = ulonlat_record[0][0]  # use the first sample's mapping
        lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
        lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}
        valid_indices = [
            (lon_to_x[lon], lat_to_y[lat])
            for lon, lat in zip(self.lons, self.lats)
            if lon in lon_to_x and lat in lat_to_y
        ]
        x_indices, y_indices = zip(*valid_indices)
        x_indices = torch.tensor(x_indices, dtype=torch.long, device=device)
        y_indices = torch.tensor(y_indices, dtype=torch.long, device=device)

        # 3D forward wavelet transform with pywt
        coeffs = self.wavelet_decompose_3d(padded_data_batch_torch)
        # Build param_vector => shape (B, 8, T/2, H/2, W/2)
        param_vector = torch.stack([
            coeffs['LLL'], coeffs['LLH'], coeffs['LHL'], coeffs['LHH'],
            coeffs['HLL'], coeffs['HLH'], coeffs['HHL'], coeffs['HHH']
        ], dim=1)

        key_to_index = {'LLL': 0, 'LLH': 1, 'LHL': 2, 'LHH': 3, 'HLL': 4, 'HLH': 5, 'HHL': 6, 'HHH': 7}
        # Suppose we only want to attack LLL, LHH => for example
        target_indices = [key_to_index[k] for k in target_key]
        non_target_indices = [i for i in range(8) if i not in target_indices]

        # Keep a copy of the original sub-bands for the non-target ones
        unchanged_param = param_vector[:, non_target_indices, :, :, :].detach().clone()

        perturbed_param_vector = param_vector.detach().clone()
        perturbed_param_vector.requires_grad_(True)

        target_torch = target.clone().detach().to(device)

        # For example, use optimizer = 'Adam'
        if use_optimizer == 'Adam':
            optimizer = torch.optim.Adam([perturbed_param_vector], lr=self.lr)

        best_loss = float('inf')
        best_perturbed_inputs = None

        for epoch in range(step_num):
            optimizer.zero_grad()
            perturbed_inputs, loss = self._forward_loss_given_params(
                perturbed_param_vector,
                inputs,
                x_indices, y_indices,
                batch_size=batch_size,
                seq_length=seq_length,
                clampEpsilon=clampEpsilon,
                mean=mean,
                std=std,
                target_torch=target_torch,
                device=device
            )

            ### regularizer
            if reg == 1:
                reg_loss = self.calculate_regularization_loss(perturbed_param_vector, param_vector, coeff_weights)
                total_loss = loss + lambda_reg * reg_loss
                print(f"Epoch {epoch} | Total Loss: {total_loss.item():.6f}, Loss: {loss.item():.6f}, reg_loss: {reg_loss:.6f}")
                total_loss.backward()
            else:
                print(f'Epoch {epoch}', ' | Loss:', loss.item(), ' | GradNorm:', torch.norm(perturbed_param_vector.grad.data).item())
                loss.backward()

            # Zero out grad for non-target sub-bands
            if perturbed_param_vector.grad is not None:
                perturbed_param_vector.grad[:, non_target_indices, :, :, :] = 0

            optimizer.step()

            # Restore non-target sub-bands
            with torch.no_grad():
                perturbed_param_vector[:, non_target_indices, :, :, :] = unchanged_param

            if loss.item() < best_loss:
                best_loss = loss.item()
                best_perturbed_inputs = perturbed_inputs

        return best_perturbed_inputs

def main():
    with open("experiments/config_clcrn_tmp2m.yaml", 'r') as f:
        clcrnConfig = yaml.load(f, Loader=yaml.SafeLoader)

    clcrnModelTrained = Supervisor(**clcrnConfig)
    clcrnModelTrained.load_model(best_CLCRN_model_epoch)
    valIterator = clcrnModelTrained._data['{}_loader'.format('test')]

    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    fable = FABLE(forward_func=clcrnModelTrained.model, wavelet='haar', level=1, lons=lons, lats=lats)

    data_list = []
    breakFlag = 0

    print(f'total implpementations: {implementations}')
    for N in range(implementations):
        print(f'Random Implementation {N}')
        for idx, (x, y) in enumerate(valIterator):
            if x.shape[0] != batch_size:
                break

            print(idx)
            # if idx == 1:
            #     break

            thisX = x.permute(1, 0, 2, 3).to('cuda')
            clcrnModelTrained.model.eval()

            if caseStudy == 0:
                predList = torch.load(f'predList_tmp2m_{N}.pth')
                targetList = torch.load(f'targetList_tmp2m_{N}.pth')
            else:
                predList = torch.load(f'case_predList_tmp2m_{N}.pth')
                targetList = torch.load(f'case_targetList_tmp2m_{N}.pth')

            yPred = predList[idx].permute(1, 0, 2, 3).to('cuda')
            yTarget = targetList[idx].permute(1, 0, 2, 3).to('cuda')

            xPerturbed = fable.perturb(thisX, yTarget, clampEpsilon, stepNum, mean, std)  # thisX.shape: (time_step, batch_size, locations, 1), e.g., [7, 32, 1320, 1]
            yPerturbed = clcrnModelTrained.model(xPerturbed.to('cuda'))

            for b in range(x.shape[0]):
                data_list.append({
                    "thisX": thisX[:, b, :, :].detach().cpu().numpy().tolist(),
                    "yPred": yPred[:, b, :, :].detach().cpu().numpy().tolist(),
                    "yTarget": yTarget[:, b, :, :].detach().cpu().numpy().tolist(),
                    "xPerturbed": xPerturbed[:, b, :, :].detach().cpu().numpy().tolist(),
                    "yPerturbed": yPerturbed[:, b, :, :].detach().cpu().numpy().tolist(),
                })

            df = pd.DataFrame(data_list)
            if caseStudy == 0:
                df.to_csv(f"results/FABLE_{dataName}_{clampEpsilon}_{stepNum}_3d_{fix}_{implementations}.csv", index=False)
            else:
                df.to_csv(f"results/FABLE_{dataName}_{clampEpsilon}_{stepNum}_3d_{fix}_caseStudy_{implementations}.csv", index=False)

            if caseStudy == 1:
                testBatchNum = 0
                if breakFlag == testBatchNum:
                    break
                breakFlag += 1

            del thisX, yPred, yTarget, xPerturbed, yPerturbed
            torch.cuda.empty_cache()
            gc.collect()

if __name__ == "__main__":
    main()


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:


2025-02-10 23:31:02,742 - INFO - Log directory: experiments/tmp2m/tmp2m


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 69

2025-02-10 23:31:02,742 - INFO - Log directory: experiments/tmp2m/tmp2m


INFO:supervisor:Log directory: experiments/tmp2m/tmp2m


...

model loaded from:  experiments/tmp2m/tmp2m/saved_model
2025-02-10 23:31:08,246 - INFO - Loaded model at 84
2025-02-10 23:31:08,246 - INFO - Loaded model at 84


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

total implpementations: 1
Random Implementation 0
0


<ipython-input-5-b13bbb68249f>:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  predList = torch.load(f'predList_tmp2m_{N}.pth')
<ipython-input-5-b13bbb68249f>:330: Future

Epoch 0 | Total Loss: 0.005281, Loss: 0.005281, reg_loss: 0.000000
Epoch 1 | Total Loss: 0.005910, Loss: 0.005898, reg_loss: 4.125345
Epoch 2 | Total Loss: 0.006391, Loss: 0.006369, reg_loss: 7.320412
Epoch 3 | Total Loss: 0.005314, Loss: 0.005287, reg_loss: 8.824853
Epoch 4 | Total Loss: 0.005543, Loss: 0.005513, reg_loss: 10.108199
Epoch 5 | Total Loss: 0.005257, Loss: 0.005225, reg_loss: 10.647511
Epoch 6 | Total Loss: 0.004539, Loss: 0.004507, reg_loss: 10.624006
Epoch 7 | Total Loss: 0.004272, Loss: 0.004240, reg_loss: 10.706548
Epoch 8 | Total Loss: 0.004295, Loss: 0.004261, reg_loss: 11.291541
Epoch 9 | Total Loss: 0.004028, Loss: 0.003991, reg_loss: 12.125437
Epoch 10 | Total Loss: 0.003667, Loss: 0.003628, reg_loss: 12.943228
Epoch 11 | Total Loss: 0.003527, Loss: 0.003486, reg_loss: 13.747566
Epoch 12 | Total Loss: 0.003400, Loss: 0.003356, reg_loss: 14.580169
Epoch 13 | Total Loss: 0.003171, Loss: 0.003125, reg_loss: 15.409531
Epoch 14 | Total Loss: 0.003027, Loss: 0.002978,

In [ ]:
# @title Visualize 2D Decomposition based on 2D Wavelet Transform on NLDAS-apcpsfc + Padding
# for NLDAS apcpsfc
# lon: 58; lat: 28

# //MARK (yue, 1/2/2025): Address the boundary issues when doing wavelet transform is a contribution. (do not speculate but see the equation and collect mathematical proof)

!pip install cartopy
!pip install torch_geometric
!pip install PyWavelets

from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import pickle
import yaml
import torch
import torch.nn.functional as F
import re
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pywt

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/CLCRN-main')

# Updated color map for precipitation-like data
colors = [(0, "white"), (0.5, "blue"), (1, "red")]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

def create_regular_padded(lons, lats, values, fill_value=0.0):
    unique_lons = sorted(list(set(lons)))
    unique_lats = sorted(list(set(lats)))
    W = len(unique_lons)
    H = len(unique_lats)

    padded_data = np.full((H, W), fill_value, dtype=np.float32)

    lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
    lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}

    for lon, lat, val in zip(lons, lats, values):
        ix = lon_to_x[lon]
        iy = lat_to_y[lat]
        padded_data[iy, ix] = val

    return padded_data, unique_lons, unique_lats

def plot_2d_map(matrix_2d, vmin, vmax, title):
    fig = plt.figure(figsize=(6, 3))
    ax = fig.add_subplot(111)
    im = ax.imshow(matrix_2d, cmap=custom_cmap, origin='lower', vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(im, orientation='vertical', pad=0.02, aspect=30)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()
    plt.title(title)
    plt.show()

def plotForecastingMap(thisData, vmin, vmax):
    lonlatAddr = "data/apcpsfc/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)

    fig_width = 6
    fig_height = 3
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES, edgecolor='black')
    ax.coastlines()
    ax.set_extent([-125, -66, 24, 50], crs=ccrs.PlateCarree())

    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    sc = ax.scatter(
        lons,
        lats,
        c=thisData,
        cmap=custom_cmap,
        marker=',',
        edgecolor='none',
        linewidth=0,
        vmin=vmin,
        vmax=vmax
    )

    cbar = plt.colorbar(sc, orientation='vertical', pad=0.02, aspect=15)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.title("Original X (Raw)", fontsize=12)
    plt.show()

def wavelet_decompose_2d(data_2d, wavelet='haar', level=1):
    coeffs_2d = pywt.wavedec2(data_2d, wavelet=wavelet, level=level)
    return coeffs_2d

def wavelet_reconstruct_2d(coeffs_2d, wavelet='haar'):
    return pywt.waverec2(coeffs_2d, wavelet=wavelet)

def plot_wavelet_components(coeffs_2d):
    cA = coeffs_2d[0]
    details = coeffs_2d[1:]

    fig, axes = plt.subplots(2, 2, figsize=(6, 3), gridspec_kw={'hspace': 0.05, 'wspace': 0.05})
    vmin = min(cA.min(), *(min(cH.min(), cV.min(), cD.min()) for cH, cV, cD in details))
    vmax = max(cA.max(), *(max(cH.max(), cV.max(), cD.max()) for cH, cV, cD in details))

    im = axes[0, 0].imshow(cA, cmap=custom_cmap, origin='lower', vmin=vmin, vmax=vmax)
    axes[0, 0].axis('off')
    cH, cV, cD = details[0]
    axes[0, 1].imshow(cH, cmap=custom_cmap, origin='lower', vmin=vmin, vmax=vmax)
    axes[0, 1].axis('off')
    axes[1, 0].imshow(cV, cmap=custom_cmap, origin='lower', vmin=vmin, vmax=vmax)
    axes[1, 0].axis('off')
    axes[1, 1].imshow(cD, cmap=custom_cmap, origin='lower', vmin=vmin, vmax=vmax)
    axes[1, 1].axis('off')

    # Shared colorbar for all components
    fig.subplots_adjust(right=0.92)
    cbar_ax = fig.add_axes([0.93, 0.15, 0.02, 0.7])
    fig.colorbar(im, cax=cbar_ax)

    plt.tight_layout()
    plt.show()

def perturb_wavelet_details(coeffs_2d, original_mask):
    random_noise_magnitude = 0.5
    perturbed_coeffs = coeffs_2d.copy()
    details = perturbed_coeffs[1:]

    current_mask = original_mask
    for i, (cH, cV, cD) in enumerate(details):
        current_mask = current_mask[::2, ::2]                                   # Downsample by 2 (Haar wavelet effect)

        cH += np.random.uniform(-random_noise_magnitude, random_noise_magnitude, size=cH.shape) * current_mask
        cV += np.random.uniform(-random_noise_magnitude, random_noise_magnitude, size=cV.shape) * current_mask
        cD += np.random.uniform(-random_noise_magnitude, random_noise_magnitude, size=cD.shape) * current_mask

        details[i] = (cH, cV, cD)

    perturbed_coeffs[1:] = details
    return perturbed_coeffs

def compare_decomposed_and_perturbed(X_perturbed_padded, perturbed_coeffs_2d, wavelet, level):
    perturbed_coeffs_2d_again = wavelet_decompose_2d(X_perturbed_padded, wavelet=wavelet, level=level)
    plot_wavelet_components(perturbed_coeffs_2d_again)

    cA_diff = np.abs(perturbed_coeffs_2d_again[0] - perturbed_coeffs_2d[0])
    details_diff = []
    for decomposed_detail, perturbed_detail in zip(perturbed_coeffs_2d_again[1:], perturbed_coeffs_2d[1:]):
        diff_cH = np.abs(decomposed_detail[0] - perturbed_detail[0])
        diff_cV = np.abs(decomposed_detail[1] - perturbed_detail[1])
        diff_cD = np.abs(decomposed_detail[2] - perturbed_detail[2])
        details_diff.append((diff_cH, diff_cV, diff_cD))

    fig, axes = plt.subplots(2, 2, figsize=(6, 3), gridspec_kw={'hspace': 0.05, 'wspace': 0.05})
    im_cA = axes[0, 0].imshow(cA_diff, cmap="viridis", origin="lower")
    axes[0, 0].axis('off')
    im_cH = axes[0, 1].imshow(details_diff[0][0], cmap="viridis", origin="lower")
    axes[0, 1].axis('off')
    im_cV = axes[1, 0].imshow(details_diff[0][1], cmap="viridis", origin="lower")
    axes[1, 0].axis('off')
    im_cD = axes[1, 1].imshow(details_diff[0][2], cmap="viridis", origin="lower")
    axes[1, 1].axis('off')
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    fig.colorbar(im_cA, cax=cbar_ax)

    plt.tight_layout()
    plt.show()

def main():
    df = pd.read_csv("results/FABLE_tmp2m_2.5_01000_2d_all.csv")
    sampleNum = 10

    X_original = torch.tensor(eval(df['thisX'][sampleNum]))[0, :, dim].cpu().numpy()
    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    vmax = X_original.max()
    vmin = X_original.min()
    plotForecastingMap(X_original, vmin, vmax)

    if padding == 'values':
        X_original_padded, unique_lons, unique_lats = create_regular_padded(lons, lats, X_original, fill_value=0.0)
    vmax = X_original_padded.max()
    vmin = X_original_padded.min()
    plot_2d_map(X_original_padded, vmin, vmax, "Original X (Padded)")

    # wavelet and level
    wavelet = 'haar'
    level = 1
    coeffs_2d = wavelet_decompose_2d(X_original_padded, wavelet=wavelet, level=level)
    plot_wavelet_components(coeffs_2d)

    mask = (X_original_padded != 0).astype(np.float32)
    perturbed_coeffs_2d = perturb_wavelet_details(coeffs_2d, mask)
    plot_wavelet_components(perturbed_coeffs_2d)

    X_perturbed_padded = wavelet_reconstruct_2d(perturbed_coeffs_2d, wavelet=wavelet)
    vmax = X_perturbed_padded.max()
    vmin = X_perturbed_padded.min()
    plot_2d_map(X_perturbed_padded, vmin, vmax, "Perturbed X (Padded)")

    # To check whether 'X_perturbed_padded', which has been reconstructed with 'perturbed_coeffs_2d', would reproduce the same 'perturbed_coeffs_2d'
    compare_decomposed_and_perturbed(X_perturbed_padded, perturbed_coeffs_2d, wavelet=wavelet, level=level)

if __name__ == '__main__':
    main()

# //TODO:
# The relationship between A, D1, D2, D3? What will happen if we add them together?
# The decomposition unique? A, D1, D2', D3? A, D1', D2, D3?... Is there any randomness we concern about?
#   e.g. X->A D1 D2 D3 -> A D1' D2 D3 -> X' -> A'' D1'' D2'' D3'', (A D1' D2 D3)=(A'' D1'' D2'' D3'')?? Do not assume the whole code will give you the correct thing
#   May be Di has some autocorrelation so that we should not perturb them independently
# How about the 2d decomposition? Check it one by one
# Do we have a way to sum up D1 D2 D3 so we can write X=A+D? Do we better perturb D or perturb Di separately?
# What is the reconstruction process in terms of 2d Wavelet decomposition? How it will affect the results?
# To understand what is the best to put them all together.


In [ ]:
# @title Visualize 3D Decomposition based on 2D Wavelet Transform on NLDAS-apcpsfc + Padding
# for NLDAS apcpsfc
# lon: 58; lat: 28

!pip install cartopy
!pip install torch_geometric
!pip install PyWavelets

import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import pickle
import torch
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

colors = [(0, "white"), (0.5, "blue"), (1, "red")]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

def create_regular_padded(lons, lats, values, fill_value=0.0):
    """
    Returns (padded_data, mask_data, unique_lons, unique_lats).
    mask_data has the same shape as padded_data, with 1 where actual data is placed,
    and 0 where fill_value is placed.
    """
    unique_lons = sorted(list(set(lons)))
    unique_lats = sorted(list(set(lats)))
    W = len(unique_lons)
    H = len(unique_lats)

    padded_data = np.full((H, W), fill_value, dtype=np.float32)
    mask_data   = np.zeros((H, W), dtype=np.float32)

    lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
    lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}

    for lon, lat, val in zip(lons, lats, values):
        ix = lon_to_x[lon]
        iy = lat_to_y[lat]
        padded_data[iy, ix] = val
        mask_data[iy, ix]   = 1.0

    return padded_data, mask_data, unique_lons, unique_lats

def plotForecastingMapMultipleSteps(data_list, vmin, vmax, title):
    lonlatAddr = "data/apcpsfc/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)

    fig_width = 6 * len(data_list)
    fig_height = 3
    fig, axes = plt.subplots(
        1, len(data_list),
        figsize=(fig_width, fig_height),
        subplot_kw={'projection': ccrs.PlateCarree()}
    )

    if len(data_list) == 1:
        axes = [axes]

    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    for idx, (thisData, ax) in enumerate(zip(data_list, axes)):
        ax.add_feature(cfeature.STATES, edgecolor='black')
        ax.coastlines()
        ax.set_extent([-125, -66, 24, 50], crs=ccrs.PlateCarree())

        sc = ax.scatter(
            lons,
            lats,
            c=thisData,
            cmap=custom_cmap,
            marker=',',
            edgecolor='none',
            linewidth=0,
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(f"{title} T{idx+1}", fontsize=10)

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(sc, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def plot_3d_map(matrix_3d, vmin, vmax, title):
    if isinstance(matrix_3d, list):
        matrix_3d = np.array(matrix_3d)

    fig_width = 6 * matrix_3d.shape[0]
    fig_height = 3
    fig, axes = plt.subplots(
        1, matrix_3d.shape[0],
        figsize=(fig_width, fig_height)
    )

    if matrix_3d.shape[0] == 1:
        axes = [axes]

    for idx, (thisData, ax) in enumerate(zip(matrix_3d, axes)):
        im = ax.imshow(thisData, cmap=custom_cmap, origin='lower',
                       vmin=vmin, vmax=vmax)
        ax.set_title(f"{title} T{idx+1}", fontsize=10)
        ax.axis('off')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def wavelet_transform_3d(data_3d, wavelet='haar'):
    """
    Perform a 2D wavelet transform over axes=(1, 2) of data shaped (Time, H, W).
    Returns a dict of sub-band coefficients.
    Example of returned keys: 'aa','ad','da','dd'
    """
    coeffs = pywt.dwtn(data_3d, wavelet=wavelet, axes=(1, 2))
    return coeffs

def wavelet_inverse_transform_3d(coeffs, wavelet='haar'):
    """
    Inverse of wavelet_transform_3d.
    Reconstructs data of shape (Time, H, W).
    """
    reconstructed_data = pywt.idwtn(coeffs, wavelet=wavelet, axes=(1, 2))
    return reconstructed_data

def plot_wavelet_coeffs(coeffs_dict, title):
    """
    Plots a dictionary of wavelet coefficients in separate figures.
    Each subband is placed in its own figure, with time slices shown as subplots.
    """
    subband_keys = list(coeffs_dict.keys())
    subband_keys.sort()

    for sb_key in subband_keys:
        subband_data = coeffs_dict[sb_key]
        vmin_value = subband_data.min()
        vmax_value = subband_data.max()

        time_len = subband_data.shape[0]

        fig_width = 4.5 * time_len
        fig_height = 4.0
        fig, axes = plt.subplots(1, time_len, figsize=(fig_width, fig_height))

        if time_len == 1:
            axes = np.array([axes])

        for t in range(time_len):
            ax = axes[t]
            im = ax.imshow(
                subband_data[t],
                cmap=custom_cmap,
                origin='lower',
                vmin=vmin_value,
                vmax=vmax_value
            )
            ax.set_title(f"{title} - {sb_key} (T{t+1})", fontsize=8)
            ax.axis('off')

        cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.5])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.locator = MaxNLocator(nbins=5)
        cbar.update_ticks()

        plt.tight_layout(rect=[0,0,0.9,1])
        plt.show()

def plot_wavelet_coeffs_difference(coeffs_dict_1, coeffs_dict_2, title):
    subband_keys = sorted(coeffs_dict_1.keys())
    time_len = coeffs_dict_1[subband_keys[0]].shape[0]

    fig_width = 4.5 * time_len
    fig_height = 9
    fig, axes = plt.subplots(len(subband_keys), time_len, figsize=(fig_width, fig_height))

    if time_len == 1:
        axes = np.expand_dims(axes, axis=1)

    for row_idx, sb_key in enumerate(subband_keys):
        arr1 = coeffs_dict_1[sb_key]
        arr2 = coeffs_dict_2[sb_key]
        diff_arr = arr1 - arr2
        for t in range(time_len):
            ax = axes[row_idx, t]
            im = ax.imshow(diff_arr[t], cmap='seismic', origin='lower')
            ax.set_title(f"{title} - {sb_key} (T{t+1})", fontsize=8)
            ax.axis('off')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def perturb_wavelet_coeffs_3d_with_mask(coeffs_dict, mask_3d, random_noise_magnitude=0.5):
    """
    Perturb only detail sub-bands ('ad','da','dd') using a provided mask_3d
    (shape: (Time, H, W)), ignoring approximation sub-band ('aa').
    The mask is downsampled by factor 2 in spatial dimensions to match sub-band shapes.
    """
    perturbed_coeffs = {}
    for sb_key, sb_data in coeffs_dict.items():
        if sb_key == 'aa':
            perturbed_coeffs[sb_key] = sb_data.copy()
            continue

        time_len, h_sub, w_sub = sb_data.shape
        # Downsample the mask in spatial dimensions by factor 2
        mask_sub = mask_3d[:time_len, ::2, ::2]

        noise = np.random.uniform(-random_noise_magnitude,
                                  random_noise_magnitude,
                                  size=sb_data.shape)
        new_data = sb_data + noise * mask_sub
        perturbed_coeffs[sb_key] = new_data

    return perturbed_coeffs

def main():
    df = pd.read_csv("results/fable_tmp2m_2.5_01000.csv")
    sampleNum = 10

    dim = 0
    X_original = torch.tensor(eval(df['thisX'][sampleNum]))[:, :, dim].cpu().numpy()
    time_steps = X_original.shape[0]
    vmin = X_original.min()
    vmax = X_original.max()

    # Read lon/lat
    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    # Plot raw data
    plotForecastingMapMultipleSteps(
        [X_original[t] for t in range(time_steps)],
        vmin, vmax,
        title="Original X (Raw)"
    )

    # Create padded data and mask
    X_padded_list = []
    mask_list = []
    for t in range(time_steps):
        X_padded, mask_padded, _, _ = create_regular_padded(
            lons, lats, X_original[t], fill_value=0.0
        )
        X_padded_list.append(X_padded)
        mask_list.append(mask_padded)

    X_original_padded = np.stack(X_padded_list)  # shape (Time, H, W)
    mask_3d = np.stack(mask_list)                # shape (Time, H, W)

    plot_3d_map(X_original_padded, vmin, vmax, "Original X (Padded)")

    # 2D wavelet transform over (H, W) for each time step
    wavelet = 'haar'
    coeffs_original = wavelet_transform_3d(X_original_padded, wavelet=wavelet)
    plot_wavelet_coeffs(coeffs_original, title="Original Coeffs")

    print(np.sum(abs(X_original_padded - pywt.idwtn(coeffs_original, wavelet = 'haar', axes = (1,2)))))

    # Perturb only detail sub-bands using mask_3d
    coeffs_perturbed = perturb_wavelet_coeffs_3d_with_mask(coeffs_original, mask_3d, random_noise_magnitude=0.5)
    plot_wavelet_coeffs(coeffs_perturbed, title="Perturbed Coeffs")

    # Reconstruct
    X_perturbed_padded = wavelet_inverse_transform_3d(coeffs_perturbed, wavelet=wavelet)
    plot_3d_map(X_perturbed_padded, vmin, vmax, "Perturbed X (Padded)")

    # Transform again
    coeffs_perturbed_again = wavelet_transform_3d(X_perturbed_padded, wavelet=wavelet)
    plot_wavelet_coeffs(coeffs_perturbed_again, title="Perturbed Coeffs Again")

    # Compare difference
    plot_wavelet_coeffs_difference(
        coeffs_perturbed, coeffs_perturbed_again,
        title="Difference"
    )

if __name__ == '__main__':
    main()


In [ ]:
# @title Visualize 3D Decomposition based on 3D Wavelet Transform on NLDAS-apcpsfc + Padding
# for NLDAS apcpsfc
# lon: 58; lat: 28

# MARK (yue, 1/11/2025): Since the Haar Wavelet Transform requires the dimension of original data is 2*, padding will be automatically implemented by 'pywt.dwtn()' if any dimension if odd.
#                        For example, if the temporal dimension is 7, the original data will be automatically padded as 8 dimension in temporal by using the last time step's data
#                        As I have tested, the original data will be perfectly reconstructed using 'pywt.idwtn()' even with the decomposed coefficients based on the padded original data
#                        NOTE: DO NOT INPUT THE ODD DIMENSIONAL DATA INTO THE FUNCTION SINCE THE AUTOMATICAL PADDING WILL MAKE THE WAVELET TRANSFORM NOT UNIQUE!!!
#                        SOLUTION: pywt.dwtn(data_3d, wavelet=wavelet, mode='zero'), where the parameter 'mode' can be specified

!pip install cartopy
!pip install torch_geometric
!pip install PyWavelets

import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import pickle
import torch
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

wavelet_mode = 'smooth' # ['smooth', 'zero', 'constant', 'symmetric', 'reflect', 'periodic', 'antisymmetric', 'antireflect']


colors = [(0, "white"), (0.5, "blue"), (1, "red")]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

def create_regular_padded(lons, lats, values, fill_value=0.0):
    """
    Create padded data and a mask of the same size.
    The mask indicates valid data (mask=1) vs. padded data (mask=0).
    """
    unique_lons = sorted(list(set(lons)))
    unique_lats = sorted(list(set(lats)))
    W = len(unique_lons)
    H = len(unique_lats)

    padded_data = np.full((H, W), fill_value, dtype=np.float32)
    mask_data = np.zeros((H, W), dtype=np.float32)

    lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}
    lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}

    for lon, lat, val in zip(lons, lats, values):
        ix = lon_to_x[lon]
        iy = lat_to_y[lat]
        padded_data[iy, ix] = val
        mask_data[iy, ix] = 1.0

    return padded_data, mask_data, unique_lons, unique_lats

def plotForecastingMapMultipleSteps(data_list, vmin, vmax, title):
    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)

    fig_width = 6 * len(data_list)
    fig_height = 3
    fig, axes = plt.subplots(
        1, len(data_list),
        figsize=(fig_width, fig_height),
        subplot_kw={'projection': ccrs.PlateCarree()}
    )

    if len(data_list) == 1:
        axes = [axes]

    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    for idx, (thisData, ax) in enumerate(zip(data_list, axes)):
        ax.add_feature(cfeature.STATES, edgecolor='black')
        ax.coastlines()
        ax.set_extent([-125, -66, 24, 50], crs=ccrs.PlateCarree())

        sc = ax.scatter(
            lons,
            lats,
            c=thisData,
            cmap=custom_cmap,
            marker=',',
            edgecolor='none',
            linewidth=0,
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(f"{title} T{idx+1}", fontsize=10)

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(sc, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()

def plot_3d_map(matrix_3d, vmin, vmax, title):
    if isinstance(matrix_3d, list):
        matrix_3d = np.array(matrix_3d)

    fig_width = 6 * matrix_3d.shape[0]
    fig_height = 3
    fig, axes = plt.subplots(
        1, matrix_3d.shape[0],
        figsize=(fig_width, fig_height)
    )

    if matrix_3d.shape[0] == 1:
        axes = [axes]

    for idx, (thisData, ax) in enumerate(zip(matrix_3d, axes)):
        im = ax.imshow(thisData, cmap=custom_cmap, origin='lower',
                       vmin=vmin, vmax=vmax)
        ax.set_title(f"{title} T{idx+1}", fontsize=10)
        ax.axis('off')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()

def wavelet_transform_3d(data_3d, wavelet='haar'):
    coeffs = pywt.dwtn(data_3d, wavelet=wavelet, mode=wavelet_mode)
    return coeffs

def wavelet_inverse_transform_3d(coeffs, wavelet='haar'):
    reconstructed_data = pywt.idwtn(coeffs, wavelet=wavelet, mode=wavelet_mode)
    return reconstructed_data

def plot_wavelet_coeffs(coeffs_dict, title):
    """
    Plots a dictionary of wavelet coefficients in separate figures.
    Each subband is placed in its own figure, with time slices shown as subplots.
    """
    subband_keys = list(coeffs_dict.keys())
    subband_keys.sort()

    for sb_key in subband_keys:
        subband_data = coeffs_dict[sb_key]
        vmin_value = subband_data.min()
        vmax_value = subband_data.max()

        time_len = subband_data.shape[0]

        fig_width = 2.0 * time_len
        fig_height = 1.5
        fig, axes = plt.subplots(1, time_len, figsize=(fig_width, fig_height))

        if time_len == 1:
            axes = np.array([axes])

        for t in range(time_len):
            ax = axes[t]
            im = ax.imshow(
                subband_data[t],
                cmap=custom_cmap,
                origin='lower',
                vmin=vmin_value,
                vmax=vmax_value
            )
            ax.set_title(f"{title} - {sb_key} (T{t+1})", fontsize=8)
            ax.axis('off')

        cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.5])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.locator = MaxNLocator(nbins=5)
        cbar.update_ticks()

        plt.tight_layout(rect=[0,0,0.9,1])
        plt.show()

def plot_wavelet_coeffs_difference(coeffs_dict_1, coeffs_dict_2, title):
    subband_keys = sorted(coeffs_dict_1.keys())
    time_len = coeffs_dict_1[subband_keys[0]].shape[0]

    fig_width = 4.5 * time_len
    fig_height = 9
    fig, axes = plt.subplots(len(subband_keys), time_len, figsize=(fig_width, fig_height))

    if time_len == 1:
        axes = np.expand_dims(axes, axis=1)

    for row_idx, sb_key in enumerate(subband_keys):
        arr1 = coeffs_dict_1[sb_key]
        arr2 = coeffs_dict_2[sb_key]
        diff_arr = abs(arr1 - arr2)
        for t in range(time_len):
            ax = axes[row_idx, t]
            im = ax.imshow(diff_arr[t], cmap='seismic', origin='lower')
            ax.set_title(f"{title} - {sb_key} (T{t+1})", fontsize=8)
            ax.axis('off')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()

def perturb_wavelet_coeffs_3d_with_mask(coeffs_dict, mask_3d, random_noise_magnitude=0.5):
    """
    Perturb wavelet coefficients in detail sub-bands using a mask.
    Approximation coefficients (key='aaa') are skipped.
    Only masked (valid) locations receive noise.
    """
    perturbed_coeffs = {}

    for sb_key, sb_data in coeffs_dict.items():
        if sb_key == 'aaa':
            perturbed_coeffs[sb_key] = sb_data.copy()
            continue

        time_len, h_sub, w_sub = sb_data.shape

        # Example: downsample the mask in spatial dimensions by factor of 2
        # You may adjust this based on actual wavelet transform axes
        mask_sub = mask_3d[:time_len, ::2, ::2]

        noise = np.random.uniform(-random_noise_magnitude,
                                  random_noise_magnitude,
                                  size=sb_data.shape)
        new_data = sb_data + noise * mask_sub
        perturbed_coeffs[sb_key] = new_data

    return perturbed_coeffs

def main():

    # read original X
    df = pd.read_csv("results/fable_tmp2m_2.5_01000.csv")
    sampleNum = 10
    dim = 0
    X_original = torch.tensor(eval(df['thisX'][sampleNum]))[:, :, dim].cpu().numpy()

    # print(X_original.shape)
    # X_original = np.concatenate([X_original, X_original[-1:]], axis=0)          # Haar Wavelet must take data with even dimensions as the input
    # print(X_original.shape)

    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    # plot original X
    time_steps = X_original.shape[0]
    vmin = X_original.min()
    vmax = X_original.max()

    plotForecastingMapMultipleSteps([X_original[t] for t in range(time_steps)], vmin, vmax, title="Original X (Raw)")

    # get and plot the padded X based on original X
    X_padded_list = []
    mask_list = []
    for t in range(time_steps):
        padded_data, mask_data, _, _ = create_regular_padded(lons, lats, X_original[t], fill_value=0.0)
        X_padded_list.append(padded_data)
        mask_list.append(mask_data)

    X_original_padded = np.stack(X_padded_list)                                 # as the input of 'pywt.dwt()' is type 'ndarray'
    mask_3d = np.stack(mask_list)

    plot_3d_map(X_original_padded, vmin, vmax, "Original X (Padded)")

    # 3d Wavelet Transform based on padded X
    wavelet = 'haar'
    coeffs_original = wavelet_transform_3d(X_original_padded, wavelet=wavelet)
    plot_wavelet_coeffs(coeffs_original, title="Original Coeffs")

    # reconstruct padded X and plot the difference between padded X and reconstructed padded X
    X_original_padded_again = wavelet_inverse_transform_3d(coeffs_original, wavelet=wavelet)
    plot_3d_map(X_original_padded_again, vmin, vmax, "Original X (Padded) Again")
    plot_3d_map(abs(X_original_padded_again[:7] - X_original_padded),vmin, vmax, "Difference on X")

    # perturb 3d Wavelet Transform components based on padded X
    coeffs_perturbed = perturb_wavelet_coeffs_3d_with_mask(coeffs_original, mask_3d, random_noise_magnitude = 0.5)
    plot_wavelet_coeffs(coeffs_perturbed, title="Perturbed Coeffs")

    # reconstruct perturbed padded X
    X_perturbed_padded = wavelet_inverse_transform_3d(coeffs_perturbed, wavelet=wavelet)
    plot_3d_map(X_perturbed_padded, vmin, vmax, "Perturbed X (Padded)")

    # 3d Wavelet Transform based on reconstructed perturbed padded X
    coeffs_perturbed_again = wavelet_transform_3d(X_perturbed_padded, wavelet=wavelet)
    plot_wavelet_coeffs(coeffs_perturbed_again, title="Perturbed Coeffs Again")

    # Compare the difference
    plot_wavelet_coeffs_difference(coeffs_perturbed, coeffs_perturbed_again, title="Difference")

if __name__ == '__main__':
    main()



In [ ]:
# @title Test My 1D Inverse Haar Wavelet Transform

!pip install PyWavelets
import pywt
import numpy as np
import torch
import math


# generate the data
Original_x_np = np.random.rand(batch_size, seq_length)
Original_x = torch.tensor(Original_x_np)

# just test one sample from the batch
sample_num = 0
sample_original_x_np = Original_x_np[sample_num, :]

### Haar wavelet Transform
# coefficients by 'pywt'
coeffs_list = pywt.dwtn(Original_x, wavelet='haar', axes=(1,))
print('a by pywt: ', coeffs_list['a'][sample_num,:])
print('d by pywt: ', coeffs_list['d'][sample_num,:])
# coefficients by me
a_by_me = []
d_by_me = []
for k in range(0, math.floor(len(sample_original_x_np)/2)):
    a_by_me.append((sample_original_x_np[2*k] + sample_original_x_np[2*k+1])/math.sqrt(2))
    d_by_me.append((sample_original_x_np[2*k] - sample_original_x_np[2*k+1])/math.sqrt(2))
print('a by me: ', a_by_me)
print('d by me: ', d_by_me)

### inverse Haar wavelet transform
# reconstruct by 'pywt'
reconstruct_x_np = pywt.idwtn(coeffs_list, wavelet='haar', axes = (1,))
print('original X: ', Original_x[sample_num, :])
print('reconstructed X by pywt: ', reconstruct_x_np[sample_num, :])
# reconstruct by me
reconstructed_x_by_me = torch.zeros(Original_x.shape)
reconstructed_x_by_me[:, ::2] = (torch.tensor(coeffs_list['a']) + torch.tensor(coeffs_list['d']))/math.sqrt(2) # g(2k)
reconstructed_x_by_me[:, 1::2] = (torch.tensor(coeffs_list['a']) - torch.tensor(coeffs_list['d']))/math.sqrt(2) # g(2k+1)
print('reconstructed X by me: ', reconstructed_x_by_me[sample_num, :])

In [ ]:
# @title Test My 2D Inverse Haar Wavelet Transform

!pip install PyWavelets
import pywt
import numpy as np
import torch
import math

# Generate random data
Original_x_np = np.random.rand(batch_size, seq_length, H, W)
Original_x = torch.tensor(Original_x_np)

# Forward Haar wavelet transform
def haar_2d_forward(x):
    B, S, H, W = x.shape
    LL = torch.zeros((B, S, H // 2, W // 2), device=x.device, dtype=x.dtype)
    LH = torch.zeros((B, S, H // 2, W // 2), device=x.device, dtype=x.dtype)
    HL = torch.zeros((B, S, H // 2, W // 2), device=x.device, dtype=x.dtype)
    HH = torch.zeros((B, S, H // 2, W // 2), device=x.device, dtype=x.dtype)

    for k_i in range(H // 2):
        for k_j in range(W // 2):
            LL[:, :, k_i, k_j] = (
                x[:, :, 2 * k_i, 2 * k_j]
                + x[:, :, 2 * k_i + 1, 2 * k_j]
                + x[:, :, 2 * k_i, 2 * k_j + 1]
                + x[:, :, 2 * k_i + 1, 2 * k_j + 1]
            ) / 2.0
            LH[:, :, k_i, k_j] = (
                x[:, :, 2 * k_i, 2 * k_j]
                + x[:, :, 2 * k_i + 1, 2 * k_j]
                - x[:, :, 2 * k_i, 2 * k_j + 1]
                - x[:, :, 2 * k_i + 1, 2 * k_j + 1]
            ) / 2.0
            HL[:, :, k_i, k_j] = (
                x[:, :, 2 * k_i, 2 * k_j]
                - x[:, :, 2 * k_i + 1, 2 * k_j]
                + x[:, :, 2 * k_i, 2 * k_j + 1]
                - x[:, :, 2 * k_i + 1, 2 * k_j + 1]
            ) / 2.0
            HH[:, :, k_i, k_j] = (
                x[:, :, 2 * k_i, 2 * k_j]
                - x[:, :, 2 * k_i + 1, 2 * k_j]
                - x[:, :, 2 * k_i, 2 * k_j + 1]
                + x[:, :, 2 * k_i + 1, 2 * k_j + 1]
            ) / 2.0

    return LL, LH, HL, HH

# Perform forward Haar wavelet transform
LL_my, LH_my, HL_my, HH_my = haar_2d_forward(Original_x.to(device))

# Perform forward Haar transform using PyWavelets
coeffs_list = pywt.dwtn(Original_x_np, wavelet='haar', axes=(2, 3))
LL_pywt = torch.tensor(coeffs_list['aa'], dtype=torch.float32).to(device)
LH_pywt = torch.tensor(coeffs_list['ad'], dtype=torch.float32).to(device)
HL_pywt = torch.tensor(coeffs_list['da'], dtype=torch.float32).to(device)
HH_pywt = torch.tensor(coeffs_list['dd'], dtype=torch.float32).to(device)

# Compare coefficients
print("Comparing coefficients:")
print("LL difference mean:", torch.abs(LL_my - LL_pywt).mean().item())
print("LH difference mean:", torch.abs(LH_my - LH_pywt).mean().item())
print("HL difference mean:", torch.abs(HL_my - HL_pywt).mean().item())
print("HH difference mean:", torch.abs(HH_my - HH_pywt).mean().item())

# Inverse Haar wavelet transform
Reconstructed_x_by_me = torch.zeros(
    (batch_size, seq_length, H, W),
    device=Original_x.device,
    dtype=Original_x.dtype
)

Reconstructed_x_by_me[:, :, ::2, ::2] = (LL_my + LH_my + HL_my + HH_my) / 2.0
Reconstructed_x_by_me[:, :, ::2, 1::2] = (LL_my - LH_my + HL_my - HH_my) / 2.0
Reconstructed_x_by_me[:, :, 1::2, ::2] = (LL_my + LH_my - HL_my - HH_my) / 2.0
Reconstructed_x_by_me[:, :, 1::2, 1::2] = (LL_my - LH_my - HL_my + HH_my) / 2.0

# Inverse using PyWavelets
Reconstructed_x_by_pywt = pywt.idwtn(coeffs_list, wavelet='haar', axes=(2, 3))

# Calculate differences
print("mean error by pywt:", (torch.abs(Original_x - torch.tensor(Reconstructed_x_by_pywt))).mean().item())
abs_diff_tensor = torch.abs(Original_x - Reconstructed_x_by_me)
print("max error by me:", abs_diff_tensor.max().item())
print("min error by me)::", abs_diff_tensor.min().item())


In [ ]:
# @title Test My 3D Inverse Haar Wavelet Transform

# NLDAS: temperature (this is for sure having it), sea level pressure (?, to check with Dr. Luo or Shane, if it has daily autocorrelation)
# do not use daily 'precipitation'(not strong enough, we used to do it by month) and 'wind speed'.
# take a time series and do the autocorrelation to check it
# six-hourly better autocorrelation?

B = batch_size
T = seq_length

def haar_3d_forward(x):
    B, T, H, W = x.shape
    inv_sqrt8 = 1.0 / math.sqrt(8.0)
    out_shape = (B, T // 2, H // 2, W // 2)

    LLL = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    LLH = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    LHL = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    LHH = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    HLL = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    HLH = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    HHL = torch.zeros(out_shape, device=x.device, dtype=x.dtype)
    HHH = torch.zeros(out_shape, device=x.device, dtype=x.dtype)

    for k_t in range(T // 2):
        for k_i in range(H // 2):
            for k_j in range(W // 2):
                X000 = x[:, 2*k_t,   2*k_i,   2*k_j   ]  # t=0,i=0,j=0
                X001 = x[:, 2*k_t,   2*k_i,   2*k_j+1 ]  # t=0,i=0,j=1
                X010 = x[:, 2*k_t,   2*k_i+1, 2*k_j   ]  # t=0,i=1,j=0
                X011 = x[:, 2*k_t,   2*k_i+1, 2*k_j+1 ]  # t=0,i=1,j=1
                X100 = x[:, 2*k_t+1, 2*k_i,   2*k_j   ]  # t=1,i=0,j=0
                X101 = x[:, 2*k_t+1, 2*k_i,   2*k_j+1 ]  # t=1,i=0,j=1
                X110 = x[:, 2*k_t+1, 2*k_i+1, 2*k_j   ]  # t=1,i=1,j=0
                X111 = x[:, 2*k_t+1, 2*k_i+1, 2*k_j+1 ]  # t=1,i=1,j=1

                LLL[:, k_t, k_i, k_j] = (X000 + X001 + X010 + X011 + X100 + X101 + X110 + X111) * inv_sqrt8
                LLH[:, k_t, k_i, k_j] = (X000 - X001 + X010 - X011 + X100 - X101 + X110 - X111) * inv_sqrt8
                LHL[:, k_t, k_i, k_j] = (X000 + X001 - X010 - X011 + X100 + X101 - X110 - X111) * inv_sqrt8
                LHH[:, k_t, k_i, k_j] = (X000 - X001 - X010 + X011 + X100 - X101 - X110 + X111) * inv_sqrt8
                HLL[:, k_t, k_i, k_j] = (X000 + X001 + X010 + X011 - X100 - X101 - X110 - X111) * inv_sqrt8
                HLH[:, k_t, k_i, k_j] = (X000 - X001 + X010 - X011 - X100 + X101 - X110 + X111) * inv_sqrt8
                HHL[:, k_t, k_i, k_j] = (X000 + X001 - X010 - X011 - X100 - X101 + X110 + X111) * inv_sqrt8
                HHH[:, k_t, k_i, k_j] = (X000 - X001 - X010 + X011 - X100 + X101 + X110 - X111) * inv_sqrt8

    return LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH

def haar_3d_inverse(LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH):
    B, T2, H2, W2 = LLL.shape
    T = T2 * 2
    H = H2 * 2
    W = W2 * 2
    x_recon = torch.zeros((B, T, H, W), device=LLL.device, dtype=LLL.dtype)
    inv_sqrt8 = 1.0 / math.sqrt(8.0)

    x_recon[:, ::2, ::2, ::2] = (LLL + LLH + LHL + LHH + HLL + HLH + HHL + HHH) * inv_sqrt8
    x_recon[:, ::2, ::2, 1::2] = (LLL - LLH + LHL - LHH + HLL - HLH + HHL - HHH) * inv_sqrt8
    x_recon[:, ::2, 1::2, ::2] = (LLL + LLH - LHL - LHH + HLL + HLH - HHL - HHH) * inv_sqrt8
    x_recon[:, ::2, 1::2, 1::2] = (LLL - LLH - LHL + LHH + HLL - HLH - HHL + HHH) * inv_sqrt8
    x_recon[:, 1::2, ::2, ::2] = (LLL + LLH + LHL + LHH - HLL - HLH - HHL - HHH) * inv_sqrt8
    x_recon[:, 1::2, ::2, 1::2] = (LLL - LLH + LHL - LHH - HLL + HLH - HHL + HHH) * inv_sqrt8
    x_recon[:, 1::2, 1::2, ::2] = (LLL + LLH - LHL - LHH - HLL - HLH + HHL + HHH) * inv_sqrt8
    x_recon[:, 1::2, 1::2, 1::2] = (LLL - LLH - LHL + LHH - HLL + HLH + HHL - HHH) * inv_sqrt8

    return x_recon

### forward
x_np = np.random.rand(B, T, H, W)

print(x_np.shape)

x = torch.tensor(x_np, dtype=torch.float32)
pywt_coeffs = pywt.dwtn(x_np, wavelet='haar', axes=(1, 2, 3))
LLL_my, LLH_my, LHL_my, LHH_my, HLL_my, HLH_my, HHL_my, HHH_my = haar_3d_forward(x)

LLL_diff = torch.abs(LLL_my - torch.tensor(pywt_coeffs['aaa'], dtype=torch.float32)).mean().item()
LLH_diff = torch.abs(LLH_my - torch.tensor(pywt_coeffs['aad'], dtype=torch.float32)).mean().item()
LHL_diff = torch.abs(LHL_my - torch.tensor(pywt_coeffs['ada'], dtype=torch.float32)).mean().item()
LHH_diff = torch.abs(LHH_my - torch.tensor(pywt_coeffs['add'], dtype=torch.float32)).mean().item()
HLL_diff = torch.abs(HLL_my - torch.tensor(pywt_coeffs['daa'], dtype=torch.float32)).mean().item()
HLH_diff = torch.abs(HLH_my - torch.tensor(pywt_coeffs['dad'], dtype=torch.float32)).mean().item()
HHL_diff = torch.abs(HHL_my - torch.tensor(pywt_coeffs['dda'], dtype=torch.float32)).mean().item()
HHH_diff = torch.abs(HHH_my - torch.tensor(pywt_coeffs['ddd'], dtype=torch.float32)).mean().item()

print("LLL difference:", LLL_diff)
print("LLH difference:", LLH_diff)
print("LHL difference:", LHL_diff)
print("LHH difference:", LHH_diff)
print("HLL difference:", HLL_diff)
print("HLH difference:", HLH_diff)
print("HHL difference:", HHL_diff)
print("HHH difference:", HHH_diff)

### inverse
coeffs = pywt.dwtn(x_np, wavelet='haar', axes=(1,2,3))
LLL = torch.tensor(coeffs['aaa'], dtype=torch.float32)
LLH = torch.tensor(coeffs['aad'], dtype=torch.float32)
LHL = torch.tensor(coeffs['ada'], dtype=torch.float32)
LHH = torch.tensor(coeffs['add'], dtype=torch.float32)
HLL = torch.tensor(coeffs['daa'], dtype=torch.float32)
HLH = torch.tensor(coeffs['dad'], dtype=torch.float32)
HHL = torch.tensor(coeffs['dda'], dtype=torch.float32)
HHH = torch.tensor(coeffs['ddd'], dtype=torch.float32)

Reconstructed_x_by_me = haar_3d_inverse(LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH)
Reconstructed_x_by_pywt = pywt.idwtn(coeffs, wavelet='haar', axes=(1,2,3))
err_by_me = (x - Reconstructed_x_by_me).abs().mean().item()
err_by_pywt = (x - Reconstructed_x_by_pywt).abs().mean().item()
diff = (Reconstructed_x_by_me - Reconstructed_x_by_pywt).abs().mean().item()

print("mean error by pywt:", err_by_pywt)
print("max error by me:", (x - Reconstructed_x_by_me).abs().max().item())
print("min error by me:", (x - Reconstructed_x_by_me).abs().min().item())

In [ ]:
# @title Analysize the perturbation magnitute added on different coefficients for 2d Haar wavelet
time_steps_shown_in_plot = 6                                                    # they are the first X time steps in X

!pip install cartopy
!pip install torch_geometric
!pip install PyWavelets

import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import pickle
import torch
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

colors = [(0, "white"), (0.5, "blue"), (1, "red")]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

def create_regular_padded(lons, lats, values, fill_value=0.0):
    """
    Returns (padded_data, mask_data, unique_lons, unique_lats).
    mask_data has the same shape as padded_data, with 1 where actual data is placed,
    and 0 where fill_value is placed.
    """
    unique_lons = sorted(list(set(lons)))
    unique_lats = sorted(list(set(lats)))
    W = len(unique_lons)
    H = len(unique_lats)

    padded_data = np.full((H, W), fill_value, dtype=np.float32)
    mask_data   = np.zeros((H, W), dtype=np.float32)

    lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}                # {-124.5: 0, -123.5: 1, -122.5: 2, -121.5: 3, -120.5: 4, -119.5: 5,  ...}
    lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}                # {25.5: 0, 26.5: 1, 27.5: 2, 28.5: 3, 29.5: 4, 30.5: 5, ...}

    for lon, lat, val in zip(lons, lats, values):
        ix = lon_to_x[lon]
        iy = lat_to_y[lat]
        padded_data[iy, ix] = val
        mask_data[iy, ix]   = 1.0

    return padded_data, mask_data, unique_lons, unique_lats

def plotForecastingMapMultipleSteps(data_list, vmin, vmax, title):
    lonlatAddr = "data/apcpsfc/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)

    fig_width = 6 * time_steps_shown_in_plot
    fig_height = 3
    fig, axes = plt.subplots(
        1, len(data_list),
        figsize=(fig_width, fig_height),
        subplot_kw={'projection': ccrs.PlateCarree()}
    )

    if len(data_list) == 1:
        axes = [axes]

    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    for idx, (thisData, ax) in enumerate(zip(data_list, axes)):
        ax.add_feature(cfeature.STATES, edgecolor='black')
        ax.coastlines()
        ax.set_extent([-125, -66, 24, 50], crs=ccrs.PlateCarree())

        sc = ax.scatter(
            lons,
            lats,
            c=thisData,
            cmap=custom_cmap,
            marker=',',
            edgecolor='none',
            linewidth=0,
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(f"{title} T{idx+1}", fontsize=10)

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(sc, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def plot_3d_map(matrix_3d, vmin, vmax, title):
    if isinstance(matrix_3d, list):
        matrix_3d = np.array(matrix_3d)

    # fig_width = 6 * matrix_3d.shape[0]
    fig_width = 6 * time_steps_shown_in_plot
    fig_height = 3
    fig, axes = plt.subplots(
        1, time_steps_shown_in_plot,
        figsize=(fig_width, fig_height)
    )

    if matrix_3d.shape[0] == 1:
        axes = [axes]

    for idx, (thisData, ax) in enumerate(zip(matrix_3d, axes)):
        if idx < time_steps_shown_in_plot:

            im = ax.imshow(thisData, cmap=custom_cmap, origin='lower',
                          vmin=vmin, vmax=vmax)
            ax.set_title(f"{title} T{idx+1}", fontsize=10)
            ax.axis('off')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def wavelet_transform_3d(data_3d, wavelet='haar'):
    """
    Perform a 2D wavelet transform over axes=(1, 2) of data shaped (Time, H, W).
    Returns a dict of sub-band coefficients.
    Example of returned keys: 'aa','ad','da','dd'
    """
    coeffs = pywt.dwtn(data_3d, wavelet=wavelet, axes=(0, 1, 2))
    return coeffs

def wavelet_inverse_transform_3d(coeffs, wavelet='haar'):
    """
    Inverse of wavelet_transform_3d.
    Reconstructs data of shape (Time, H, W).
    """
    reconstructed_data = pywt.idwtn(coeffs, wavelet=wavelet, axes=(0, 1, 2))
    return reconstructed_data

def plot_wavelet_coeffs(coeffs_dict, title):
    """
    Plots a dictionary of wavelet coefficients in separate figures.
    Each subband is placed in its own figure, with time slices shown as subplots.
    """
    subband_keys = list(coeffs_dict.keys())
    subband_keys.sort()

    for sb_key in subband_keys:
        subband_data = coeffs_dict[sb_key]
        vmin_value = subband_data.min()
        vmax_value = subband_data.max()

        # time_len = subband_data.shape[0]


        fig_width = 4.5 * time_steps_shown_in_plot
        fig_height = 4.0
        fig, axes = plt.subplots(1, time_steps_shown_in_plot, figsize=(fig_width, fig_height))

        if time_steps_shown_in_plot == 1:
            axes = np.array([axes])

        for t in range(time_steps_shown_in_plot):
            ax = axes[t]
            im = ax.imshow(
                subband_data[t],
                cmap=custom_cmap,
                origin='lower',
                vmin=vmin_value,
                vmax=vmax_value
            )
            ax.set_title(f"{title} - {sb_key} (T{t+1})", fontsize=8)
            ax.axis('off')

        cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.5])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.locator = MaxNLocator(nbins=5)
        cbar.update_ticks()

        plt.tight_layout(rect=[0,0,0.9,1])
        plt.show()

def main():
    df = pd.read_csv("results/FABLE_tmp2m_2.5_01000_2d_LL+HL+HH.csv")
    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]
    dim = 0

    LL_diff = []
    LH_diff = []
    HL_diff = []
    HH_diff = []
    LL = []
    LH = []
    HL = []
    HH = []

    for sampleNum in range(len(df['thisX'])):
        # print(sampleNum)

        ## original
        # print('original')
        X_original = torch.tensor(eval(df['thisX'][sampleNum]))[:, :, dim].cpu().numpy()
        time_steps = X_original.shape[0]
        vmin = X_original.min()
        vmax = X_original.max()
        X_padded_list = []
        mask_list = []
        for t in range(time_steps):
            X_padded, mask_padded, _, _ = create_regular_padded(lons, lats, X_original[t], fill_value=0.0)
            X_padded_list.append(X_padded)
            mask_list.append(mask_padded)
        X_original_padded = np.stack(X_padded_list)
        mask_3d = np.stack(mask_list)
        # plot_3d_map(X_original_padded, vmin, vmax, "Original X (Padded)")
        wavelet = 'haar'
        coeffs_original = wavelet_transform_3d(X_original_padded, wavelet=wavelet)
        # plot_wavelet_coeffs(coeffs_original, title="Original Coeffs")

        # ## perturbed
        # print('perturbed')
        X_perturbed = torch.tensor(eval(df['xPerturbed'][sampleNum]))[:, :, dim].cpu().numpy()
        time_steps = X_perturbed.shape[0]
        vmin = X_perturbed.min()
        vmax = X_perturbed.max()
        X_perturbed_padded_list = []
        mask_list = []
        for t in range(time_steps):
            X_perturbed_padded, mask_padded, _, _ = create_regular_padded(
                lons, lats, X_perturbed[t], fill_value=0.0
            )
            X_perturbed_padded_list.append(X_perturbed_padded)
            mask_list.append(mask_padded)
        X_perturbed_padded = np.stack(X_perturbed_padded_list)
        mask_3d = np.stack(mask_list)
        # plot_3d_map(X_perturbed_padded, vmin, vmax, "Perturbed X (Padded)")
        wavelet = 'haar'
        coeffs_perturbed = wavelet_transform_3d(X_perturbed_padded, wavelet=wavelet)
        # plot_wavelet_coeffs(coeffs_perturbed, title="Perturbed Coeffs")

        ## diff
        # print('diff')
        X_diff = X_original - X_perturbed
        time_steps = X_diff.shape[0]
        vmin = X_diff.min()
        vmax = X_diff.max()
        X_diff_padded_list = []
        mask_list = []
        for t in range(time_steps):
            X_diff_padded, mask_padded, _, _ = create_regular_padded(
                lons, lats, X_diff[t], fill_value=0.0
            )
            X_diff_padded_list.append(X_diff_padded)
            mask_list.append(mask_padded)
        X_diff_padded = np.stack(X_diff_padded_list)
        mask_3d = np.stack(mask_list)
        # plot_3d_map(X_diff_padded, vmin, vmax, "Diff X (Padded)")
        coeffs_diff = {}
        for key in coeffs_original.keys():
            coeffs_diff[key] = coeffs_original[key] - coeffs_perturbed[key]
        # plot_wavelet_coeffs(coeffs_diff, title="Coeffs diff")


        ###
        LL.append(np.mean(abs(coeffs_original['aa'])))
        LH.append(np.mean(abs(coeffs_original['ad'])))
        HL.append(np.mean(abs(coeffs_original['da'])))
        HH.append(np.mean(abs(coeffs_original['dd'])))

        LL_diff.append(np.mean(abs(coeffs_diff['aa'])))
        LH_diff.append(np.mean(abs(coeffs_diff['ad'])))
        HL_diff.append(np.mean(abs(coeffs_diff['da'])))
        HH_diff.append(np.mean(abs(coeffs_diff['dd'])))

    # print(np.mean(LL), np.std(LL))
    # print(np.mean(LH), np.std(LH))
    # print(np.mean(HL), np.std(HL))
    # print(np.mean(HH), np.std(HH))

    print(np.mean(LL_diff), np.std(LL_diff))
    print(np.mean(LH_diff), np.std(LH_diff))
    print(np.mean(HL_diff), np.std(HL_diff))
    print(np.mean(HH_diff), np.std(HH_diff))


if __name__ == '__main__':
    main()


In [ ]:
# @title Analysize the perturbation magnitute added on different coefficients for 3d Haar wavelet
time_steps_shown_in_plot = 6                                                    # they are the first X time steps in X

!pip install cartopy
!pip install torch_geometric
!pip install PyWavelets

import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import pickle
import torch
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

colors = [(0, "white"), (0.5, "blue"), (1, "red")]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

def create_regular_padded(lons, lats, values, fill_value=0.0):
    """
    Returns (padded_data, mask_data, unique_lons, unique_lats).
    mask_data has the same shape as padded_data, with 1 where actual data is placed,
    and 0 where fill_value is placed.
    """
    unique_lons = sorted(list(set(lons)))
    unique_lats = sorted(list(set(lats)))
    W = len(unique_lons)
    H = len(unique_lats)

    padded_data = np.full((H, W), fill_value, dtype=np.float32)
    mask_data   = np.zeros((H, W), dtype=np.float32)

    lon_to_x = {val: idx for idx, val in enumerate(unique_lons)}                # {-124.5: 0, -123.5: 1, -122.5: 2, -121.5: 3, -120.5: 4, -119.5: 5,  ...}
    lat_to_y = {val: idx for idx, val in enumerate(unique_lats)}                # {25.5: 0, 26.5: 1, 27.5: 2, 28.5: 3, 29.5: 4, 30.5: 5, ...}

    for lon, lat, val in zip(lons, lats, values):
        ix = lon_to_x[lon]
        iy = lat_to_y[lat]
        padded_data[iy, ix] = val
        mask_data[iy, ix]   = 1.0

    return padded_data, mask_data, unique_lons, unique_lats

def plotForecastingMapMultipleSteps(data_list, vmin, vmax, title):
    lonlatAddr = "data/apcpsfc/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)

    fig_width = 6 * time_steps_shown_in_plot
    fig_height = 3
    fig, axes = plt.subplots(
        1, len(data_list),
        figsize=(fig_width, fig_height),
        subplot_kw={'projection': ccrs.PlateCarree()}
    )

    if len(data_list) == 1:
        axes = [axes]

    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]

    for idx, (thisData, ax) in enumerate(zip(data_list, axes)):
        ax.add_feature(cfeature.STATES, edgecolor='black')
        ax.coastlines()
        ax.set_extent([-125, -66, 24, 50], crs=ccrs.PlateCarree())

        sc = ax.scatter(
            lons,
            lats,
            c=thisData,
            cmap=custom_cmap,
            marker=',',
            edgecolor='none',
            linewidth=0,
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(f"{title} T{idx+1}", fontsize=10)

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(sc, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def plot_3d_map(matrix_3d, vmin, vmax, title):
    if isinstance(matrix_3d, list):
        matrix_3d = np.array(matrix_3d)

    # fig_width = 6 * matrix_3d.shape[0]
    fig_width = 6 * time_steps_shown_in_plot
    fig_height = 3
    fig, axes = plt.subplots(
        1, time_steps_shown_in_plot,
        figsize=(fig_width, fig_height)
    )

    if matrix_3d.shape[0] == 1:
        axes = [axes]

    for idx, (thisData, ax) in enumerate(zip(matrix_3d, axes)):
        if idx < time_steps_shown_in_plot:

            im = ax.imshow(thisData, cmap=custom_cmap, origin='lower',
                          vmin=vmin, vmax=vmax)
            ax.set_title(f"{title} T{idx+1}", fontsize=10)
            ax.axis('off')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()

    plt.tight_layout(rect=[0,0,0.9,1])
    plt.show()

def wavelet_transform_3d(data_3d, wavelet='haar'):
    """
    Perform a 2D wavelet transform over axes=(1, 2) of data shaped (Time, H, W).
    Returns a dict of sub-band coefficients.
    Example of returned keys: 'aa','ad','da','dd'
    """
    coeffs = pywt.dwtn(data_3d, wavelet=wavelet)
    return coeffs

def wavelet_inverse_transform_3d(coeffs, wavelet='haar'):
    """
    Inverse of wavelet_transform_3d.
    Reconstructs data of shape (Time, H, W).
    """
    reconstructed_data = pywt.idwtn(coeffs, wavelet=wavelet)
    return reconstructed_data

def plot_wavelet_coeffs(coeffs_dict, title):
    """
    Plots a dictionary of wavelet coefficients in separate figures.
    Each subband is placed in its own figure, with time slices shown as subplots.
    """
    subband_keys = list(coeffs_dict.keys())
    subband_keys.sort()

    for sb_key in subband_keys:
        subband_data = coeffs_dict[sb_key]
        vmin_value = subband_data.min()
        vmax_value = subband_data.max()

        # time_len = subband_data.shape[0]


        fig_width = 4.5 * time_steps_shown_in_plot
        fig_height = 4.0
        fig, axes = plt.subplots(1, time_steps_shown_in_plot, figsize=(fig_width, fig_height))

        if time_steps_shown_in_plot == 1:
            axes = np.array([axes])

        for t in range(time_steps_shown_in_plot):
            ax = axes[t]
            im = ax.imshow(
                subband_data[t],
                cmap=custom_cmap,
                origin='lower',
                vmin=vmin_value,
                vmax=vmax_value
            )
            ax.set_title(f"{title} - {sb_key} (T{t+1})", fontsize=8)
            ax.axis('off')

        cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.5])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.locator = MaxNLocator(nbins=5)
        cbar.update_ticks()

        plt.tight_layout(rect=[0,0,0.9,1])
        plt.show()

def main():
    df = pd.read_csv("results/FABLE_tmp2m_2.5_01000_3d_2L1H_HHH.csv")
    lonlatAddr = "data/tmp2m/position_info.pkl"
    with open(lonlatAddr, 'rb') as lonLatPkl:
        lonLatDict = pickle.load(lonLatPkl)
    lons = lonLatDict['lonlat'][:, 0]
    lats = lonLatDict['lonlat'][:, 1]
    dim = 0

    LLL_diff = []
    LLH_diff = []
    LHL_diff = []
    LHH_diff = []
    HLL_diff = []
    HLH_diff = []
    HHL_diff = []
    HHH_diff = []
    LLL = []
    LLH = []
    LHL = []
    LHH = []
    HLL = []
    HLH = []
    HHL = []
    HHH = []


    for sampleNum in range(len(df['thisX'])):
        # print(sampleNum)

        ## original
        # print('original')
        X_original = torch.tensor(eval(df['thisX'][sampleNum]))[:, :, dim].cpu().numpy()
        time_steps = X_original.shape[0]
        vmin = X_original.min()
        vmax = X_original.max()
        X_padded_list = []
        mask_list = []
        for t in range(time_steps):
            X_padded, mask_padded, _, _ = create_regular_padded(lons, lats, X_original[t], fill_value=0.0)
            X_padded_list.append(X_padded)
            mask_list.append(mask_padded)
        X_original_padded = np.stack(X_padded_list)
        mask_3d = np.stack(mask_list)
        # plot_3d_map(X_original_padded, vmin, vmax, "Original X (Padded)")
        wavelet = 'haar'
        coeffs_original = wavelet_transform_3d(X_original_padded, wavelet=wavelet)
        # plot_wavelet_coeffs(coeffs_original, title="Original Coeffs")

        # ## perturbed
        # print('perturbed')
        X_perturbed = torch.tensor(eval(df['xPerturbed'][sampleNum]))[:, :, dim].cpu().numpy()
        time_steps = X_perturbed.shape[0]
        vmin = X_perturbed.min()
        vmax = X_perturbed.max()
        X_perturbed_padded_list = []
        mask_list = []
        for t in range(time_steps):
            X_perturbed_padded, mask_padded, _, _ = create_regular_padded(
                lons, lats, X_perturbed[t], fill_value=0.0
            )
            X_perturbed_padded_list.append(X_perturbed_padded)
            mask_list.append(mask_padded)
        X_perturbed_padded = np.stack(X_perturbed_padded_list)
        mask_3d = np.stack(mask_list)
        # plot_3d_map(X_perturbed_padded, vmin, vmax, "Perturbed X (Padded)")
        wavelet = 'haar'
        coeffs_perturbed = wavelet_transform_3d(X_perturbed_padded, wavelet=wavelet)
        # plot_wavelet_coeffs(coeffs_perturbed, title="Perturbed Coeffs")

        ## diff
        # print('diff')
        X_diff = X_original - X_perturbed
        time_steps = X_diff.shape[0]
        vmin = X_diff.min()
        vmax = X_diff.max()
        X_diff_padded_list = []
        mask_list = []
        for t in range(time_steps):
            X_diff_padded, mask_padded, _, _ = create_regular_padded(
                lons, lats, X_diff[t], fill_value=0.0
            )
            X_diff_padded_list.append(X_diff_padded)
            mask_list.append(mask_padded)
        X_diff_padded = np.stack(X_diff_padded_list)
        mask_3d = np.stack(mask_list)
        # plot_3d_map(X_diff_padded, vmin, vmax, "Diff X (Padded)")
        coeffs_diff = {}
        for key in coeffs_original.keys():
            coeffs_diff[key] = coeffs_original[key] - coeffs_perturbed[key]
        # plot_wavelet_coeffs(coeffs_diff, title="Coeffs diff")


        ###
        LLL.append(np.mean(abs(coeffs_original['aaa'])))
        LLH.append(np.mean(abs(coeffs_original['aad'])))
        LHL.append(np.mean(abs(coeffs_original['ada'])))
        LHH.append(np.mean(abs(coeffs_original['add'])))
        HLL.append(np.mean(abs(coeffs_original['daa'])))
        HLH.append(np.mean(abs(coeffs_original['dad'])))
        HHL.append(np.mean(abs(coeffs_original['dda'])))
        HHH.append(np.mean(abs(coeffs_original['ddd'])))

        LLL_diff.append(np.mean(abs(coeffs_diff['aaa'])))
        LLH_diff.append(np.mean(abs(coeffs_diff['aad'])))
        LHL_diff.append(np.mean(abs(coeffs_diff['ada'])))
        LHH_diff.append(np.mean(abs(coeffs_diff['add'])))
        HLL_diff.append(np.mean(abs(coeffs_diff['daa'])))
        HLH_diff.append(np.mean(abs(coeffs_diff['dad'])))
        HHL_diff.append(np.mean(abs(coeffs_diff['dda'])))
        HHH_diff.append(np.mean(abs(coeffs_diff['ddd'])))

    # print(np.mean(LLL), np.std(LLL))
    # print(np.mean(LLH), np.std(LLH))
    # print(np.mean(LHL), np.std(LHL))
    # print(np.mean(LHH), np.std(LHH))
    # print(np.mean(HLL), np.std(HLL))
    # print(np.mean(HLH), np.std(HLH))
    # print(np.mean(HHL), np.std(HHL))
    # print(np.mean(HHH), np.std(HHH))

    print(np.mean(LLL_diff), np.std(LLL_diff))
    print(np.mean(LLH_diff), np.std(LLH_diff))
    print(np.mean(LHL_diff), np.std(LHL_diff))
    print(np.mean(LHH_diff), np.std(LHH_diff))
    print(np.mean(HLL_diff), np.std(HLL_diff))
    print(np.mean(HLH_diff), np.std(HLH_diff))
    print(np.mean(HHL_diff), np.std(HHL_diff))
    print(np.mean(HHH_diff), np.std(HHH_diff))



if __name__ == '__main__':
    main()


In [6]:
# @title Implement evaluation
dim = 0 # dim=0 (or 1 only for 'WeatherBench-surface wind components')

!pip install pysal esda libpysal

from google.colab import drive
import pandas as pd
import numpy as np
import os
import torch
import pickle

from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
from esda.moran import Moran
import statsmodels.api as sm
from scipy.stats import zscore
from libpysal.weights import DistanceBand, block_weights
import copy

dataName = 'tmp2m'
with_concatenate = 0

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/CLCRN-main')

if wavelet_decomposition_dimension == 2:
    fileName = f"FABLE_tmp2m_{clampEpsilon}_{stepNum}_2d_{fix}_{implementations}"
elif wavelet_decomposition_dimension == 3:
    fileName = f"FABLE_tmp2m_{clampEpsilon}_{stepNum}_3d_{fix}_{implementations}"

df = pd.read_csv('results/' + fileName + ".csv")

gPrecision = []
inPrecision = []
outPrecision = []
closeness = []
sMoranDiffAvgX = []
tAutoDiffAvgX = []

sMoranDiffConcatX = []
tAutoDiffConcatX = []

sMoranXListAll = []
sMoranXAdvListAll = []

def spatialMoranI(X, W):
    moran = Moran(X, W)

    return moran.I

def temporalAutocorrelation(timeSeries):
    acfValues = sm.tsa.acf(timeSeries)

    if not np.isnan(acfValues).any():
        return acfValues
    else:
        return np.zeros(len(acfValues))

def expand_lonlat(lonlat, T, delta=1000):
    N = lonlat.shape[0]
    lonlat_expanded = np.tile(lonlat, (T, 1))  # shape: (N*T, 2)

    for t in range(T):
        lonlat_expanded[t*N:(t+1)*N, 1] += t * delta

    return lonlat_expanded

lonlatAddr = 'data/' + dataName + '/position_info.pkl'
with open(lonlatAddr, 'rb') as lonLatPkl:
    lonLatDict = pickle.load(lonLatPkl)
lonlat = lonLatDict['lonlat']
lonlat_expanded = expand_lonlat(lonlat, 12)

threshold = 1                                                                   # This is a hyperparameter
W = DistanceBand(lonlat, threshold=threshold, binary=False, silence_warnings=True) # w.n: 1320*1320
if with_concatenate == 1:
    W_concat = DistanceBand(lonlat_expanded, threshold=threshold, binary=False, silence_warnings=True)

print(len(df['thisX']))
for sampleNum in range(len(df['thisX'])): # a list of tensors
    print(sampleNum)

    ### used for DEBUG
    # if sampleNum == 48:
    #     break

    ### in-target precision
    # format of 'inPositions': tensor([[timestep, location], [], ..., []])
    inPositions =  torch.nonzero(
        torch.tensor(eval(df['yTarget'][sampleNum]))[:, :, dim]
        - torch.tensor(eval(df['yPred'][sampleNum]))[:, :, dim])
    thisInPrecision = torch.sum(
        torch.abs(
            torch.tensor(eval(df['yTarget'][sampleNum]))[inPositions[:, 0], inPositions[:, 1], dim]
            - torch.tensor(eval(df['yPerturbed'][sampleNum]))[inPositions[:, 0], inPositions[:, 1], dim])
        )
    inPrecision.append(thisInPrecision.item())

    ### out-target precision.
    outPositions = torch.nonzero(
        (torch.tensor(eval(df['yTarget'][sampleNum]))[:, :, dim]
        - torch.tensor(eval(df['yPred'][sampleNum]))[:, :, dim])==0)
    thisOutPrecision = torch.sum(
        torch.abs(
            torch.tensor(eval(df['yTarget'][sampleNum]))[outPositions[:, 0], outPositions[:, 1], dim]
            - torch.tensor(eval(df['yPerturbed'][sampleNum]))[outPositions[:, 0], outPositions[:, 1], dim])
        )
    outPrecision.append(thisOutPrecision.item())

    ### closeness
    thisCloseness = torch.mean(
        torch.abs(torch.tensor(eval(df['thisX'][sampleNum]))[:, :, dim]
        - torch.tensor(eval(df['xPerturbed'][sampleNum]))[:, :, dim]))
    closeness.append(thisCloseness.item())

    ### spatial autocorrelation-averaged
    X = torch.tensor(eval(df['thisX'][sampleNum]))[:, :, dim].numpy()           # X.shape: 1320*T
    XAdv = torch.tensor(eval(df['xPerturbed'][sampleNum]))[:, :, dim].numpy()
    sMoranXList = [spatialMoranI(X[i,:].flatten(), W) for i in range(X.shape[0])] # for each element in the list, the shape is (1320,)
    sMoranXAdvList = [spatialMoranI(XAdv[i,:].flatten(), W) for i in range(XAdv.shape[0])]
    sMoranXDiffList = [abs(a - b) for a, b in zip(sMoranXList, sMoranXAdvList)]
    thisSMoranDiffAvgX = np.sum(sMoranXDiffList)
    sMoranDiffAvgX.append(thisSMoranDiffAvgX)
    sMoranXListAll.append(sMoranXList)
    sMoranXAdvListAll.append(sMoranXAdvList)

    ### spatial autocorrelation-concatenated
    if with_concatenate == 1:
        X_concat = X.flatten()
        XAdv_concat = XAdv.flatten()
        moran_all_X = Moran(X_concat, W_concat)
        moran_all_XAdv = Moran(XAdv_concat, W_concat)
        thisSMoranDiffConcatX = abs(moran_all_X.I - moran_all_XAdv.I)
        sMoranDiffConcatX.append(thisSMoranDiffConcatX)

    ### temporal autocorrelation-averaged
    tAutoXList = [temporalAutocorrelation(X[:,i]) for i in range(X.shape[1])]   # For NLDAS-precipitation dataset, there are some n.a. outcomes for temporal autocorrelation. So, we first replace them as all zeros and then take those invalid out
    tAutoXNp = np.array(tAutoXList)                                             # len(tAutoXList): 1320. tAutoXNp.shape: (1320, T)
    tAutoXAdvList = [temporalAutocorrelation(XAdv[:,i]) for i in range(XAdv.shape[1])]
    tAutoXAdvNp = np.array(tAutoXAdvList)
    tAutoDiffNp = np.abs(tAutoXNp - tAutoXAdvNp)                                # tAutoDiffNp.shape: (1320, T)
    thisTAutoDiffAvgX = np.mean(tAutoDiffNp, axis=1)
    thisTAutoDiffAvgX = np.sum(thisTAutoDiffAvgX)
    tAutoDiffAvgX.append(thisTAutoDiffAvgX)

    ### temporal autocorrelation-concatenated
    if with_concatenate == 1:
        X_concat    = X.flatten()                                                   # shape: (N*T,)
        XAdv_concat = XAdv.flatten()                                                # shape: (N*T,)
        acf_X    = temporalAutocorrelation(X_concat)
        acf_XAdv = temporalAutocorrelation(XAdv_concat)
        acf_diff = np.abs(acf_X - acf_XAdv)
        thisTAutoDiffConcatX = np.sum(acf_diff)
        tAutoDiffConcatX.append(thisTAutoDiffConcatX)

    if with_concatenate == 1:
        print(thisInPrecision.item(), thisOutPrecision.item(), thisCloseness.item(), thisSMoranDiffAvgX, thisSMoranDiffConcatX, thisTAutoDiffAvgX, thisTAutoDiffConcatX)
    else:
        print(thisInPrecision.item(), thisOutPrecision.item(), thisCloseness.item(), thisSMoranDiffAvgX, thisTAutoDiffAvgX)

print(np.mean(inPrecision), np.std(inPrecision))
print(np.mean(outPrecision), np.std(outPrecision))
print(np.mean(closeness), np.std(closeness))
print(np.mean(sMoranDiffAvgX), np.std(sMoranDiffAvgX))
print(np.mean(tAutoDiffAvgX), np.std(tAutoDiffAvgX))

if with_concatenate == 1:
    print(np.mean(sMoranDiffConcatX), np.std(sMoranDiffConcatX))
    print(np.mean(tAutoDiffConcatX), np.std(tAutoDiffConcatX))

if with_concatenate == 1:
    dict = {
        'In-Target Precision': inPrecision,
        'Out-Target Precision': outPrecision,
        'Closeness': closeness,
        'Spatial Autocorrelation X Averaged': sMoranDiffAvgX,
        'Spatial Autocorrelation X Concatenated': sMoranDiffConcatX,
        'Temporal Autocorrelation X Averaged': tAutoDiffAvgX,
        'Temporal Autocorrelation X Concatenated': tAutoDiffConcatX,
        'Moran X': sMoranXListAll,
        'Moran XAdv': sMoranXAdvListAll
    }
else:
    dict = {
      'In-Target Precision': inPrecision,
      'Out-Target Precision': outPrecision,
      'Closeness': closeness,
      'Spatial Autocorrelation X Averaged': sMoranDiffAvgX,
      'Temporal Autocorrelation X Averaged': tAutoDiffAvgX,
      'Moran X': sMoranXListAll,
      'Moran XAdv': sMoranXAdvListAll
    }

df = pd.DataFrame(dict)
df.to_csv('results/' + fileName + "_evaluation" + ".csv", index = False)


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.6/141.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/scipy/sparse/_data.py:133: RuntimeWarning: divide by zero encountered in reciprocal
  return self._with_data(data ** n)


96
0
0.41244611144065857 3.234973192214966 0.004845290444791317 0.09295616510137938 15.31489821805663
1
0.4756087362766266 5.137279033660889 0.006605035159736872 0.1126303458528205 17.35967700445132
2
0.46411845088005066 4.807016849517822 0.0052484008483588696 0.11867170937686833 20.030145238013347
3
0.4109136164188385 1.8713587522506714 0.004753814544528723 0.07353335109748516 14.004296591346023
4
0.4574815332889557 2.1204352378845215 0.006328877527266741 0.15942173560237372 18.163600598456103
5
0.43217960000038147 3.619762897491455 0.005394412204623222 0.1320820239490239 16.253458495174357
6
0.42654043436050415 2.1800849437713623 0.006932769902050495 0.1588270256383646 20.865191616641322
7
0.42983168363571167 2.1762712001800537 0.006385400891304016 0.20187627314401457 23.051021066221715
8
0.4283880591392517 3.2283921241760254 0.006746976636350155 0.21023120641420479 19.8738785977948
9
0.44991153478622437 3.997163772583008 0.007419520057737827 0.29695198016152613 21.904517181174498
10